# **Installs**

In [ ]:
!pip install photutils --quiet
!pip install webbpsf --quiet

In [ ]:
!wget https://stsci.box.com/shared/static/qxpiaxsjwo15ml6m4pkhtk36c9jgj70k.gz -O webbpsf-data-1.2.1.tar.gz

In [ ]:
!tar -xf webbpsf-data-1.2.1.tar.gz

In [ ]:
!rm webbpsf-data-1.2.1.tar.gz

In [ ]:
!pip uninstall numexpr --quiet -y

# **Imports**

In [ ]:
from gloess import gloess

In [ ]:
import sys
import time
import json
import numpy as np
import pandas as pd
import glob as glob
import urllib.request
import tarfile
import os
import itertools

from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d

from astropy.io import fits
from astropy.visualization import simple_norm
from astropy.nddata import NDData
from astropy.wcs import WCS
from astropy.modeling import models, fitting
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.table import Table, QTable, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky, angular_separation
from astropy import units as u
from astropy.stats import SigmaClip, sigma_clipped_stats, gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel, convolve_fft
from astropy.nddata.utils import Cutout2D

import photutils as phot
from photutils.background import MMMBackground, MADStdBackgroundRMS, Background2D, LocalBackground
from photutils.detection import DAOStarFinder
from photutils.psf import EPSFBuilder, GriddedPSFModel
from photutils.psf import SourceGrouper, extract_stars, IterativePSFPhotometry, DAOPhotPSFPhotometry

from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, EllipticalAperture

from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde
from scipy.ndimage import sobel, gaussian_filter, convolve
from scipy.signal import savgol_filter
from astropy.stats.biweight import biweight_location, biweight_midvariance,biweight_scale
from matplotlib.colors import LinearSegmentedColormap
from scipy.signal import find_peaks

In [ ]:
from reproject import reproject_exact
from reproject.mosaicking import find_optimal_celestial_wcs

In [ ]:
os.environ['WEBBPSF_PATH'] = '../data/webbpsf-data'

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.colors as col
from matplotlib.colors import ListedColormap
from matplotlib.patches import Circle
import seaborn as sb
import matplotlib.gridspec as gridspec
sb.set_style('white')
from matplotlib.ticker import (MultipleLocator, AutoLocator, AutoMinorLocator)
from matplotlib.patches import Rectangle

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (7,5)
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 25
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 25

font1 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '15'}
font2 = {'family': 'sans-serif', 'color': 'black', 'weight': 'normal', 'size': '25'}

In [ ]:
import webbpsf
from webbpsf.utils import to_griddedpsfmodel

In [ ]:
""" Simulated (Webb PSF)
filters = ['F070W', 'F090W', 'F115W', 'F140M', 'F150W2','F150W', 'F162M', 'F164N', 'F182M',
           'F187N', 'F200W', 'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2','F323N',
           'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W', 'F460M', 'F466N', 'F470N', 'F480M']

psf_fwhm = [0.987,   1.103,   1.298,   1.553,   1.628,   1.770,   1.801,   1.494,   1.990, 
            2.060,   2.141,   2.304,   2.341,   1.340,   1.444,   1.585,   1.547,   1.711, 
            1.760,   1.830,   1.901,   2.165,   2.179,   2.300,   2.302,   2.459,   2.507, 2.535, 2.574]

dict_utils = {filters[i]: {'psf fwhm': psf_fwhm[i]} for i in range(len(filters))}
""";

In [ ]:
# Empirical (Rigby et al. 2022). https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-performance/nircam-point-spread-functions
filters = ['F070W', 'F090W', 'F115W', 'F140M', 'F150W2','F150W', 'F162M', 'F164N', 'F182M',
           'F187N', 'F200W', 'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2','F323N',
           'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W', 'F460M', 'F466N', 'F470N', 'F480M']

psf_fwhm = [0.935,   1.065,   1.290,   1.548,   1.628,   1.613,   1.774,   1.806,   2.000, 
            2.065,   2.129,   2.290,   2.323,   1.349,   1.460,   1.587,   1.547,   1.714, 
            1.762,   1.841,   1.905,   2.159,   2.175,   2.286,   2.302,   2.492,   2.508, 2.540, 2.603]

dict_utils = {filters[i]: {'psf fwhm': psf_fwhm[i]} for i in range(len(filters))}

# **Synthetic PSF using Webb PSF**

In [ ]:
psfs_dir = 'PSF_MODELS/'

if not os.path.exists(psfs_dir):
    os.makedirs(psfs_dir)

def create_psf_model(det='NRCA1', filt='F770W', fov=101, source=None,
                     create_grid=False, num=9, save_psf=False,
                     detsampled=False,all_detectors=False,oversample=5):

    nrc = webbpsf.NIRCam()

    nrc.detector = det
    nrc.filter = filt

    print("Using a {field}".format(field=fov), "px fov")

    if create_grid:
        print("")
        print("Creating a grid of PSF for filter {filt} and detector {det}".format(filt=filt, det=det))
        print("")
        num = num

        if save_psf:

            outname = 'PSF_%s_samp%s_fov%d_npsfs%d.fits' % (filt,oversample, fov, num)
            psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=source, all_detectors=all_detectors,fov_pixels=fov,
                               save=True, outfile=os.path.join(psfs_dir,outname), use_detsampled_psf=detsampled)

        else:

            psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=source, all_detectors=all_detectors, fov_pixels=fov,
                               use_detsampled_psf=detsampled)

    else:
        print("")
        print("Creating a single PSF for filter {filt} and detector {det}".format(filt=filt, det=det))
        print("")
        num = 1

        if save_psf:

            outname = 'PSF_%s_samp%s_fov%d_npsfs%d.fits' % (filt, oversample, fov, num)
            psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=source, all_detectors=False, fov_pixels=fov,
                               save=True, outfile=os.path.join(psfs_dir,outname), use_detsampled_psf=detsampled)

        else:

            psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=source, all_detectors=False, fov_pixels=fov,
                               use_detsampled_psf=detsampled)

    return psf

In [ ]:

psf_webbpsf_single = create_psf_model(det=det_n, filt=filt_n, fov=51, source=None, create_grid=False, save_psf=False,
                                      detsampled=False)

In [ ]:
filts = [i.split('_')[2] for i in glob.glob(os.path.join(psfs_dir, "*fits"))]
if filt_n not in filts:
    psf_webbpsf_grid = create_psf_model(det=det_n, filt=filt_n, fov=51, source=None, create_grid=True, num=9,
                                        save_psf=True, detsampled=False, all_detectors=False, oversample=5)
else:
    psf_filename = glob.glob(os.path.join('PSF_MODELS/',f'PSF_{filt_n}*{det_n.lower()}.fits'))
    
    with fits.open(psf_filename) as hdu:
        psf_webbpsf_grid = to_griddedpsfmodel(hdu, ext_data=0, ext_header=0)

In [ ]:
webbpsf.gridded_library.display_psf_grid(psf_webbpsf_grid, zoom_in=False, figsize=(14, 10))

In [ ]:
def calc_bkg(data, sigma=3.0, box_size=(100,100), filter_size=(3,3)):

    bkgrms = MADStdBackgroundRMS()
    mmm_bkg = MMMBackground()

    #sigma_clip = SigmaClip(sigma=sigma)
    sigma_clip = SigmaClip(sigma=sigma)
    coverage_mask = (data == 0)

    bkg = Background2D(data, box_size, filter_size=filter_size,
                       sigma_clip=sigma_clip, bkg_estimator=mmm_bkg,
                        coverage_mask=coverage_mask, fill_value=0.0)

    data_bkgsub = data.copy()
    data_bkgsub = data_bkgsub - bkg.background

    _, _, std = sigma_clipped_stats(data_bkgsub)

    return data_bkgsub, bkg, std

In [ ]:
def find_stars(data, det='NRCA1', filt='F070W', threshold=3, sigma=3.0, sigma_psf=None, smoothen=True, kernel=None,ZP=0):

    if sigma_psf is None:
        sigma_psf = dict_utils[filt]['psf fwhm']

    if smoothen and kernel is not None:
        data = convolve_fft(data, kernel)

    data_bkgsub, bkg, std = calc_bkg(data, sigma)

    daofind = DAOStarFinder(threshold=threshold*std, fwhm=sigma_psf)
    found_stars = daofind(data_bkgsub)
    found_stars['mag'] = ZP + found_stars['mag']

    print('\nNumber of sources found in the image:', len(found_stars))
    print('-------------------------------------\n')

    return found_stars, data_bkgsub,bkg

# **Input Data**

### **JWST**

In [ ]:
dict_images = {'NRCA1': {}, 'NRCA2': {}, 'NRCA3': {}, 'NRCA4': {}, 'NRCA5': {},
               'NRCB1': {}, 'NRCB2': {}, 'NRCB3': {}, 'NRCB4': {}, 'NRCB5': {}}

dict_cats = {}

images_dir = '../data/NGC_628/JWST/'

images = sorted(glob.glob(os.path.join(images_dir, "*/*i2d.fits")))
cats  = sorted(glob.glob(os.path.join(images_dir, "*/*cat.ecsv")))

for image in images:

    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']

    if d == 'MULTIPLE':
        d = 'NRCB3'

    # Image
    if d not in dict_images.keys():
        dict_images[d] = {}
        
    if f not in dict_images[d].keys():
        dict_images[d][f] =  {'images': [image]}
    else:
        dict_images[d][f]['images'].append(image)
    
    # Catalog
    if d not in dict_cats.keys():
        dict_cats[d] = {}
        
    if f not in dict_cats[d].keys():
        dict_cats[d][f] =  {'cats': [image[:-8] + 'cat.ecsv']}
    else:
        dict_cats[d][f]['cats'].append(image[:-8] + 'cat.ecsv')
        
 
print('Available Detectors and Filters\n-------------------------------')
      
for i in dict_images.keys():
    if len(dict_images[i])>0:
        print(f'{i} :', list(dict_images[i].keys()))

In [ ]:
# MIRI
det_m = 'MIRIMAGE'
filt_m = 'F770W'

hdul_m = fits.open(dict_images[det_m][filt_m]['images'][0])
data_m = hdul_m[1].data
area_m = hdul_m[4].data

imh_m = hdul_m[1].header
wcs_m = WCS(imh_m)
ZAB_m = -6.1 - 2.5*np.log10(imh_m['PIXAR_SR'])
hdul_m.close()
print(f"Flux units of {filt_m}: {imh_m['BUNIT']}, Zero point (AB) : {ZAB_m}")
# NIRCAM
det_n = 'NRCB3'
filt_n = 'F200W'

hdul_n = fits.open(dict_images[det_n][filt_n]['images'][0])
data_n = hdul_n[1].data

imh_n = hdul_n[1].header
wcs_n = WCS(imh_n)
ZAB_n = -6.1 -2.5*np.log10(imh_n['PIXAR_SR'])

print(f"Flux units of {filt_n}: {imh_n['BUNIT']}, Zero point (AB) : {ZAB_n}")
hdul_n.close()

In [ ]:
"""
wcs_new = wcs_m.deepcopy()

wcs_new.wcs.pc= [[-1,0],
                 [0,1]]
wcs_new.wcs.crpix = np.array([750,750])
wcs_new.wcs.crval= np.array([24.173953, 15.783691])

out, footprint= reproject_exact((data_m, wcs_m), wcs_new, (1500,1500))
""";

In [ ]:
"""
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection=wcs_new)

norm = simple_norm(out, log_a=100, stretch= 'log',min_cut=12.6, max_cut = 65)

img = ax.imshow(out, norm = norm , cmap='gray')
cb = plt.colorbar(img, shrink=0.75, orientation='horizontal', pad=0.09)
cb.set_label('MJy/Sr')
cb.ax.tick_params(labelsize=20)


pos_bub = wcs_new.world_to_pixel_values([[24.186079, 15.772189]])
pos_gal = wcs_new.world_to_pixel_values([[24.173953, 15.783691]])

aper_in = EllipticalAperture(pos_bub, 162.26, 127.11, 216*u.degree)
aper_out = EllipticalAperture(pos_bub, 90.15, 70.62, 216*u.degree)

aper_in.plot(ax)
aper_out.plot(ax)

ax.annotate('', xy=pos_bub[0],
             xycoords='data',
             xytext=pos_gal[0],
             textcoords='data',
             arrowprops=dict(arrowstyle= '<|-|>',
                             color='white',
                             lw=1.5,
                             ls='-')
           )

ax.text(450,502,"1'=2.8kpc",rotation=46, color='white', fontsize=20)

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=7,direction="in")
ax.tick_params(which='minor', length=4)

lon = ax.coords[0]
lat = ax.coords[1]

lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
ax.set_xlabel('Right Ascension')
ax.set_ylabel('Declination')
""";

In [ ]:
"""
plt.figure(figsize=(20, 12))

ax1= plt.subplot(1, 2, 1, projection=wcs_m)

ax1.set_title(filt_m, fontdict=font2)
norm = simple_norm(data_m, 'sqrt',min_cut=12, max_cut = 40)

img = ax1.imshow(data_m, norm = norm, cmap='gray')
ax1.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
cb.set_label(f"{imh_m['BUNIT']}")

ax2 = plt.subplot(1, 2, 2, projection=wcs_n)

ax2.set_title(filt_n, fontdict=font2)
norm = simple_norm(data_n, 'sqrt',percent=99.)


img = ax2.imshow(data_n, norm = norm, cmap='gray')
ax2.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
cb.set_label(f"{imh_n['BUNIT']}")
""";

### **Photometry strategy**

In [ ]:
ps = imh_n['CDELT1']*3600

In [ ]:
l = 24/ps

In [ ]:
l

In [ ]:
y = np.arange(0, 4640 + l, l)
x = np.arange(0, 11604 + l, l)

x_cen = 0.5*(x[:-1] + x[1:])
y_cen = 0.5*(y[:-1] + y[1:])

In [ ]:
len(x_cen), len(y_cen)

In [ ]:
poss = []

x_offset = 0
y_offset = 0
for i in x_cen:
    for j in y_cen:
        poss.append([i-x_offset,j-y_offset])
        
poss = np.array(poss)

In [ ]:
x = poss[:,0]
y = poss[:,1]
fig, ax = plt.subplots(figsize=(12,4))
ax.scatter(x,y)

for i,j in poss:
        ax.add_patch(Rectangle((i-l/2, j-l/2), l,l, fill = False,
                              edgecolor='black', lw=1))

ax.set_xlim(0,11604)
ax.set_ylim(0,4640)

In [ ]:
coords  = np.array(wcs_n.pixel_to_world_values(poss)).astype(np.float64)

In [ ]:
coords_cen  = np.array(wcs_n.pixel_to_world_values([[0,7*20/ps]]))

In [ ]:
ra_cen, dec_cen = coords_cen[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

x = coords[:,0]
y = coords[:,1]
ax.scatter(x,y)

regions = {}
for n,coord in enumerate(coords):
    i = coord[0]
    j = coord[1]
    regions[f'reg_{n}'] = {}
    regions[f'reg_{n}']['ra'] = i
    regions[f'reg_{n}']['dec'] = j
    print(f"""box({np.round(i,9)}, {np.round(j,9)}, {l*ps}", {l*ps}",245.00492) # text=""" + '{' + f'Reg {n}' + '}')

ax.add_patch(Rectangle((ra_cen, dec_cen), 18*20/3600,7*20/3600, fill = False,
                              edgecolor='black', lw=1, angle=180-245.00492))

ra_t = regions['reg_0']['ra']
dec_t = regions['reg_0']['dec']

ax.scatter([ra_t], [dec_t], marker='x', color='black')
ax.invert_xaxis()
ax.axis('equal')

In [ ]:
with open('regions90.json', 'w') as convert_file: 
     convert_file.write(json.dumps(regions))

#### **Radial Completeness**

In [ ]:
poss_cen = wcs_n.world_to_pixel_values([[ra,dec]])[0]

In [ ]:
poss_cen[0]

In [ ]:
poss = []
for i in range(5):
    poss.append([poss_cen[0]+1307.*i,poss_cen[1]])

poss = np.array(poss)

In [ ]:
x = poss[:,0]
y = poss[:,1]
fig, ax = plt.subplots(figsize=(12,4))
ax.scatter(x,y)

for i,j in poss:
        ax.add_patch(Rectangle((i-653.5, j-653.5), 1307,1307, fill = False,
                              edgecolor='black', lw=1))


ax.set_xlim(0,11417)
ax.set_ylim(0,4486)

In [ ]:
940/20

In [ ]:
coords = np.array(wcs_n.pixel_to_world_values(poss))

In [ ]:
coords[0,1]

In [ ]:
r = angular_separation(coords[:,0]*u.deg, coords[:,1]*u.deg,
                                         coords[0,0]*u.deg, coords[0,1]*u.deg).to(u.arcsec).value*47/1e3

In [ ]:
f115w_c = [24.3, 25.02, 25.76, 26.33, 27.06]
f150w_c = [23.45, 24.17, 25.01, 25.51, 25.88]
f200w_c = [22.3,  23.03, 23.95, 24.58, 25.04]

In [ ]:
fig,ax = plt.subplots(figsize=(7,5))

ax.plot(r, f115w_c,'-o')
ax.plot(r, f150w_c,'-o')
ax.plot(r, f200w_c,'-o')
ax.legend(['F115W','F150W','F200W'])
ax.set_xlabel('r [Mpc]')
ax.set_ylabel(r'$m_{50}$')

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

#ax.set_yticks(np.arange(22.5,27,0.5))
ax.tick_params(which='both', length=14,direction="in", bottom=True, top=True,left=True, labelsize=12)
ax.tick_params(which='minor', length=8)
ax.set_title(r'Radial Compleness limit variation')

In [ ]:
regions = {}
for n,coord in enumerate(coords):
    i = coord[0]
    j = coord[1]
    regions[f'reg_{n}'] = {}
    regions[f'reg_{n}']['ra'] = i
    regions[f'reg_{n}']['dec'] = j

In [ ]:
regions

### **Plots**

In [ ]:
pos = SkyCoord(ra=ra, dec=dec, unit='deg')

In [ ]:
cutout_m = Cutout2D(data_m,pos,size = 41*u.arcsec, wcs = wcs_m)
cutout_n = Cutout2D(data_n,pos,size = 41*u.arcsec, wcs = wcs_n)

In [ ]:
cutout_n.data.shape

In [ ]:
plt.figure(figsize=(32, 20))

ax1 = plt.subplot(1, 2, 1, projection=cutout_m.wcs)

ax1.set_title(filt_m, fontdict=font2)
norm = simple_norm(cutout_m.data, 'sqrt',min_cut=13, max_cut = 40)

img = ax1.imshow(cutout_m.data, norm = norm, cmap='gray')
# MIRI Contours
levels = [13.2,15]
cmap = ListedColormap(['yellow','cyan'])
smooth_data = gaussian_filter(cutout_m.data, 2)
ax1.contour(smooth_data, levels=levels, cmap=cmap)

#ax1.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.6))
cb.set_label(f"{imh_m['BUNIT']}")

ax2 = plt.subplot(1, 2, 2, projection=cutout_n.wcs)

ax2.set_title(filt_n, fontdict=font2)
norm = simple_norm(cutout_n.data, 'log',min_percent=10, max_percent=99., log_a =10)

img = ax2.imshow(cutout_n.data, norm = norm, cmap='gray')
ax2.contour(smooth_data, levels=levels, cmap=cmap, transform=ax2.get_transform(cutout_m.wcs))
#ax2.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.6))
cb.set_label(f"{imh_n['BUNIT']}")

#plt.savefig('MIRI_NIRCAM_bubble.eps', bbox_inches='tight')

In [ ]:
cutout_n.data.shape

### **HST**

In [ ]:
images_dir = '../data/NGC_628/HST/'
images = sorted(glob.glob(os.path.join(images_dir, "*/*drc.fits")))

dict_images_hst = {}

for image in images:

    im = fits.open(image)
    if 'FILTER2' in im[0].header.keys():
        f = im[0].header['FILTER2']
    elif 'FILTER' in im[0].header.keys():
        f = im[0].header['FILTER']
    
    if 'CLEAR' in f:
        f = im[0].header['FILTER1']
        
    d = im[0].header['DETECTOR']

    # Image
    if d not in dict_images_hst.keys():
        dict_images_hst[d] = {}
        
    if f not in dict_images_hst[d].keys():
        dict_images_hst[d][f] =  {'images': [image]}
    else:
        dict_images_hst[d][f]['images'].append(image)    
 
print('Available Detectors and Filters\n-------------------------------')
      
for i in dict_images_hst.keys():
    if len(dict_images_hst[i])>0:
        print(f'{i} :', list(dict_images_hst[i].keys()))

In [ ]:
det_n = 'WFC'
filt_n = 'F814W'

hdul_n = fits.open(dict_images_hst[det_n][filt_n]['images'][0])
data_n = hdul_n[1].data

imh_n = hdul_n[1].header
wcs_n = WCS(imh_n)

print(f"Flux units of {filt_n}: {imh_n['BUNIT']}")
hdul_n.close()

In [ ]:
pos = SkyCoord(ra=ra, dec=dec, unit='deg')

In [ ]:
cutout_m = Cutout2D(data_m,pos,size = 50*u.arcsec, wcs = wcs_m)
cutout_n = Cutout2D(data_n,pos,size = 50*u.arcsec, wcs = wcs_n)

In [ ]:
plt.figure(figsize=(30, 20))

ax1 = plt.subplot(1, 2, 1, projection=cutout_m.wcs)

ax1.set_title(filt_m, fontdict=font2)
norm = simple_norm(cutout_m.data, 'sqrt',min_cut=13, max_cut = 40)

img = ax1.imshow(cutout_m.data, norm = norm, cmap='gray')
# MIRI Contours
levels = [13.2,15]
cmap = ListedColormap(['yellow','cyan'])
smooth_data = gaussian_filter(cutout_m.data, 2)
ax1.contour(smooth_data, levels=levels, cmap=cmap)

ax1.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
cb.set_label(f"{imh_m['BUNIT']}")

ax2 = plt.subplot(1, 2, 2, projection=cutout_n.wcs)

ax2.set_title(filt_n, fontdict=font2)
norm = simple_norm(cutout_n.data, 'sqrt',percent=99.)

img = ax2.imshow(cutout_n.data, norm = norm, cmap='gray')
ax2.contour(smooth_data, levels=levels, cmap=cmap, transform=ax2.get_transform(cutout_m.wcs))
ax2.grid(color='red', ls='solid')
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.5))
cb.set_label(f"{imh_n['BUNIT']}")

### **PSF**

#### **Synthetic PSF**

# **Photometric Analysis**

## **Functions**

In [ ]:
def rotate_coords(x,y,h=0,k=0,theta=0,Sx=1,Sy=1):
    
    X = np.array([x,y,x**0])

    T = np.array([[1,0,h],
                 [0,1,k],
                 [0,0,1]])

    Tinv =  np.array([[1,0,-h],
                      [0,1,-k],
                      [0,0,1]])

    R = np.array([[np.cos(theta), -np.sin(theta),0],
                  [np.sin(theta),  np.cos(theta),0],
                  [0            ,              0,1]])
    
    S = np.array([[Sx,0,0],
                  [0, Sy,0],
                  [0,0,1]])
    
    X_n = T@S@R@Tinv@X
    
    return X_n[0], X_n[1]

In [ ]:
def gen_CMD(filt1='f115w', filt2='f150w', filt3=None,name=None,r=333, r_in=None, r_out=None, mag_err_lim=0.2,
             out_dir='PHOT_OUTPUT_EPSF',Av=0.19,  Av_ = 3, Av_x=2, Av_y=19, sqr_field=False, ab_dist = True,
            dismod=29.95, xlims=[-0.5,2.5], ylims=[18,28], label=3, met=0.02, gen_contours=False,
            cmd=None, regions=['bubble','bkg1', 'bkg2','bkg3'], mag_err_cols = ['mag_err_1','mag_err_2'],
            gen_kde=False, add_ref=False, add_ext='', fig = None, ax= None, label_min=None, label_max=None,
             ra_col = 'ra_1', dec_col= 'dec_1', ages=[7.,8.,9.], alpha=1, age_Z = False,s =0.2,
            comp_flag=False, ref_xpos=-0.25):
    
    if label_min is None or label_max is None:
        label_min=0
        label_max=label
    if filt3 is None:
        filt3 = filt2
    if met is not None and cmd is not None:
        cmd = cmd[cmd['Zini']==met]
        
    age_lin = []
    for i in ages:
        if i<6:
            age_lin.append(f'{np.ceil(10**i)} Myr')
        if i >= 6  and i <9:
            i-=6
            age_lin.append(f'{np.ceil(10**i)} Myr')
        elif i >= 9:
            i-=9
            age_lin.append(f'{np.ceil(10**i)} Gyr')
            
    if fig is None:        
        fig = plt.figure(figsize=(len(regions)*12,10))
        
    for i,region in enumerate(regions):
        if ax is None:
            ax = fig.add_subplot(1,len(regions),int(i+1))
        
        if name is None:
            tab_bub = Table.read(f"../{out_dir}/{region}/{filt1}_{filt2}{add_ext}.fits")
        else:
            tab_bub = Table.read(f"../{out_dir}/{name}.fits")
            
        for i in mag_err_cols:
            tab_bub = tab_bub[tab_bub[i]<=mag_err_lim]

        AF1 =  Av_dict[filt1]*Av
        AF2 =  Av_dict[filt2]*Av
        AF3 =  Av_dict[filt3]*Av
        
        ra_cen = regions_dict[region]['ra']
        dec_cen = regions_dict[region]['dec']
        
        ang = 245.00492
        ra_new = (tab_bub[ra_col]-ra_cen)*np.cos(np.pi/180*ang) - (tab_bub[dec_col]-dec_cen)*np.sin(np.pi/180*ang) + ra_cen
        dec_new = (tab_bub[ra_col]-ra_cen)*np.sin(np.pi/180*ang) + (tab_bub[dec_col]-dec_cen)*np.cos(np.pi/180*ang) + dec_cen
        
        tab_bub['temp_ra'] = tab_bub[ra_col]
        tab_bub['temp_dec'] = tab_bub[dec_col] 
        tab_bub[ra_col] = ra_new
        tab_bub[dec_col] = dec_new
        
        tab_bub['r'] = angular_separation(tab_bub[ra_col]*u.deg,tab_bub[dec_col]*u.deg,
                                          ra_cen*u.deg, dec_cen*u.deg).to(u.arcsec).value
        
        if (r_out is None or r_in is None) :
            r_in = 0
            r_out = r
            
        if r_out is not None:  
            if not sqr_field:
                tab_bub = tab_bub[ (tab_bub['r']>=r_in) & (tab_bub['r'] <r_out)]
            else:
                tab_bub = tab_bub[ (tab_bub[ra_col]>=ra_cen-r_out/3600) & (tab_bub[ra_col]<=ra_cen+r_out/3600) ]
                tab_bub = tab_bub[ (tab_bub[dec_col]>=dec_cen-r_out/3600) & (tab_bub[dec_col]<=dec_cen+r_out/3600) ]
        
        tab_bub[ra_col] = tab_bub['temp_ra']
        tab_bub[dec_col] = tab_bub['temp_dec']
        tab_bub.remove_columns(['temp_ra', 'temp_dec'])
        
        x = tab_bub[f'mag_vega_{filt1.upper()}'] - tab_bub[f'mag_vega_{filt2.upper()}']
        y = tab_bub[f'mag_vega_{filt3.upper()}'] 
        
        x = x.value.astype(float)
        y = y.value.astype(float)
        
        n_sources = len(x)
        
        if gen_kde and not gen_contours:
            # Peform the kernel density estimate
            xx, yy = np.mgrid[xlims[0]:xlims[1]:100j, ylims[0]:ylims[1]:100j]
            positions = np.vstack([xx.ravel(), yy.ravel()])
            values = np.vstack([x, y])

            kernel = gaussian_kde(values, bw_method=0.05)
            f = np.reshape(kernel(positions), xx.shape)

            f = f.T
            img = ax.imshow(f, cmap='gray_r', 
                          extent=[xlims[0], xlims[1], 
                                  ylims[0], ylims[1]],
                           interpolation='nearest', aspect='auto')

            
        elif gen_contours:
            cmap_custom = LinearSegmentedColormap.from_list("custom_grey_to_white", ["grey", "white"], N=256)
            ax.scatter(x,y, s=0.2, color='black', label='data')
            sb.kdeplot(x=x, y=y, levels=[0.1, 0.25,0.5,0.75,0.9,0.95,0.99],
                       ax=ax, fill=True, cmap=cmap_custom)
            sb.kdeplot(x=x, y=y, levels=[0.1, 0.25,0.5,0.75,0.9,0.95,0.99],
                       ax=ax,  color='black')
            
            
        else:
            ax.scatter(x,y, s=s, color='black', label='data')
         
        if ab_dist:
            ax1 = ax.twinx()  # instantiate a second axes that shares the same x-axis

            ax1.set_ylabel(r'$M_{' + f'{filt3.upper()}' + r'}$')  # we already handled the x-label with ax1

        ax.set_xlabel(f"{filt1.upper()} - {filt2.upper()}")
        ax.set_ylabel(filt3.upper())
        
        # Error bars
        ref = tab_bub[f'mag_vega_{filt2.upper()}']
        ref_new = np.arange(np.ceil(y.min()),np.ceil(y.max()) + 0.5,0.5)
        
        mag_err1 = tab_bub[mag_err_cols[0]]
        mag_err2 = tab_bub[mag_err_cols[1]]
        
        if len(mag_err_cols)>2:
            mag_err3 = tab_bub[mag_err_cols[2]]
        else:
            mag_err3 = mag_err2

        col_err = np.sqrt(mag_err1**2 + mag_err2**2)
        
        init = models.Exponential1D()
        fit = fitting.LevMarLSQFitter()
        model_col = fit(init,ref,col_err)
        
        init = models.Exponential1D()
        fit = fitting.LevMarLSQFitter()
        model_mag = fit(init,ref,mag_err3)
        
        x = ref_xpos + 0*ref_new
        y = ref_new
        yerr = model_mag(ref_new)
        xerr = model_col(ref_new)

        ax.errorbar(x, y, yerr,xerr ,fmt='o', color = 'red', markersize=0.5, capsize=2) 
        
        if cmd is not None:
            for i, age in enumerate(ages):
                t = cmd[np.round(cmd['logAge'],1)==age].copy()
                
                mags = np.unique(t['Zini'])
                
                for Z in mags:
                    t_ = t[t['Zini']==Z]
                    t_ = t_[(t_['label']>=label_min) & (t_['label']<=label_max)]

                    x =  (t_[f'{filt1.upper()}mag'] + AF1) - (t_[f'{filt2.upper()}mag'] + AF2)
                    y =  t_[f'{filt3.upper()}mag'] + AF3 + dismod

                    mask = (y.values[1:]- y.values[:-1])<1
                    mask = np.array([True] + list(mask))
                    mask = np.where(~mask, np.nan, 1)

                    if len(mags)>1:
                        ax.plot(x*mask, y*mask, linewidth=3, label=age_lin[i]+ f' {Z}', alpha=0.6)
                    else:
                        ax.plot(x*mask, y*mask, linewidth=3, label=age_lin[i], alpha=alpha)
                
        else: 
            met = ' '

        ax.set_ylim(ylims[0], ylims[1])
        ax.set_xlim(xlims[0], xlims[1]) 
        
        yticks = ax.get_yticks()
        yticks_n = yticks - dismod
        
        dy = yticks_n - np.floor(yticks_n)
        if ab_dist:
            ax1.set_yticks(yticks + dy, np.floor(yticks_n))

            ax1.set_ylim(ylims[0], ylims[1])
            ax1.set_xlim(xlims[0], xlims[1]) 

        ax.invert_yaxis()
        if ab_dist:
            ax1.invert_yaxis()
        ax.set_title(region.capitalize() + f" | No of sources: {n_sources} | Z : {met}", fontsize=30)
            
        if add_ref:
            ax.plot([xlims[0], xlims[1]],[25,25], '--r', zorder=200)
            ax.plot([xlims[0], xlims[1]],[24.5,24.5], '--r', zorder=200)
            ax.plot([xlims[0], xlims[1]],[23,23], '--r', zorder=200)
         
        if comp_flag:
            comp_f1 = comp[region][filt1]
            comp_f2 = comp[region][filt2]

            y0 = comp_f2
            x0 = comp_f1 - comp_f2

            ax.plot([x0,xlims[1]],[y0,y0])
            
        ax.legend(fontsize=15)
        ax.xaxis.set_major_locator(AutoLocator())
        ax.xaxis.set_minor_locator(AutoMinorLocator())

        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        

        ax.tick_params(which='both', length=14,direction="in", bottom=True, top=True,left=True)
        ax.tick_params(which='minor', length=8)
        
        if ab_dist:
            ax1.yaxis.set_minor_locator(AutoMinorLocator())
            ax1.tick_params(which='both', length=14,direction="in", right=True)
            ax1.tick_params(which='minor', length=8)

        AF1_ =  Av_dict[filt1]*Av_
        AF2_ =  Av_dict[filt2]*Av_
        AF3_ =  Av_dict[filt3]*Av_

        dx = AF1_ - AF2_
        dy = AF3_

        ax.annotate('', xy=(Av_x, Av_y),
                     xycoords='data',
                     xytext=(Av_x+dx, Av_y+dy),
                     textcoords='data',
                     arrowprops=dict(arrowstyle= '<|-',
                                     color='black',
                                     lw=0.5,
                                     ls='-')
                   )
            
        ax.annotate(f'Av = {Av_}', xy=(Av_x-0.1, Av_y-0.1))
        
    fig.tight_layout()
    return fig, ax, tab_bub

In [ ]:
def gen_CMD_xcut( filt1='f115w', filt2='f150w',filt3=None,region='bubble', 
                 name=None, Av=0.19, r=333, sqr_field=False,
                 r_in=None, r_out=None, met=0.02, mag_err_cols = ['mag_err_1','mag_err_2'],
                 dismod=29.95, fit_slope=False, cmd_ylo=None, cmd_yhi=None,
                 mag_err_lim=0.2,label_min=0, label_max=10,
                 cmd=None, ra_col = 'ra_1', dec_col= 'dec_1',
                out_dir='PHOT_OUTPUT_EPSF', xlims=[-0.5,2.5], ylims=[18,30],
                age=9.0, cmd_xlo = None, cmd_xhi= None, age_Z=False,
                l_lo = 22, l_hi=26.5, dl=0.5, Av_ = 3,
                Av_x=2, Av_y=22, flag=111, fit_isochrone=True, fig=None, ax=None):
        
    if filt3 is None:
        filt3 = filt2
        
    if met is not None:
        cmd = cmd[cmd['Zini']==met]
        
    if cmd_ylo is None or cmd_yhi is None:
        cmd_ylo = l_lo - dl
        cmd_yhi = l_hi + dl
    
    if name is None:
        tab_bub = Table.read(f"../{out_dir}/{region}/{filt1}_{filt2}.fits")
    else:
        tab_bub = Table.read(f"../{out_dir}/{name}.fits")
        
    for i in mag_err_cols:
        tab_bub = tab_bub[tab_bub[i]<=mag_err_lim]
    
    AF1 =  Av_dict[filt1]*Av
    AF2 =  Av_dict[filt2]*Av
    AF3 =  Av_dict[filt3]*Av

    ra_cen = regions_dict[region]['ra']
    dec_cen = regions_dict[region]['dec']
    
    ang = 245.00492
    ra_new = (tab_bub[ra_col]-ra_cen)*np.cos(np.pi/180*ang) - (tab_bub[dec_col]-dec_cen)*np.sin(np.pi/180*ang) + ra_cen
    dec_new = (tab_bub[ra_col]-ra_cen)*np.sin(np.pi/180*ang) + (tab_bub[dec_col]-dec_cen)*np.cos(np.pi/180*ang) + dec_cen

    tab_bub['temp_ra'] = tab_bub[ra_col] 
    tab_bub['temp_dec'] = tab_bub[dec_col] 
    tab_bub[ra_col] = ra_new
    tab_bub[dec_col] = dec_new

    tab_bub['r'] = angular_separation(tab_bub[ra_col]*u.deg,tab_bub[dec_col]*u.deg,
                                      ra_cen*u.deg, dec_cen*u.deg).to(u.arcsec).value

    if (r_out is None or r_in is None) and r is not None:
        r_in = 0
        r_out = r
        
    if r_out is not None:    
        if not sqr_field:
            tab_bub = tab_bub[ (tab_bub['r']>=r_in) & (tab_bub['r'] < r_out)]
        else:
            tab_bub = tab_bub[ (tab_bub[ra_col]>=ra_cen-r_out/3600) & (tab_bub[ra_col]<=ra_cen+r_out/3600) ]
            tab_bub = tab_bub[ (tab_bub[dec_col]>=dec_cen-r_out/3600) & (tab_bub[dec_col]<=dec_cen+r_out/3600) ]

    tab_bub[ra_col] = tab_bub['temp_ra']
    tab_bub[dec_col] = tab_bub['temp_dec']
    tab_bub.remove_columns(['temp_ra', 'temp_dec'])
    
    x = tab_bub[f'mag_vega_{filt1.upper()}'] - tab_bub[f'mag_vega_{filt2.upper()}']
    y = tab_bub[f'mag_vega_{filt3.upper()}'] 

    x = x.value.astype(float)
    y = y.value.astype(float)

    if cmd_xlo is None or cmd_xhi is None:
        cmd_xlo = x.mean() - 0.5

        cmd_xhi = x.mean() + 0.5
    
    if fig is None or ax is None:
        fig = plt.figure(figsize=(7,7))
        ax = fig.add_subplot(1,1,1)
    
    theta = np.arctan(AF3/(AF1-AF2)) #
    
    ax.scatter(x,y, s=0.1, color='black')
    
    if cmd is not None:
        cmd = cmd[(cmd['label']>=label_min) & (cmd['label']<=label_max)]

        t = cmd[np.round(cmd['logAge'],1)==age].copy()

        met = np.array(cmd['Zini'])[0]
        x_i =  (t[f'{filt1.upper()}mag'] + AF1) - (t[f'{filt2.upper()}mag'] + AF2)
        y_i =  t[f'{filt3.upper()}mag'] + AF3 + dismod
        
        x_i = np.array(x_i)
        y_i = np.array(y_i)
        
        x_iso = x_i.copy()
        y_iso = y_i.copy()
        
        x_i = x_i[np.where( (y_i>=cmd_ylo) & (y_i<=cmd_yhi))[0]]
        y_i = y_i[np.where( (y_i>=cmd_ylo) & (y_i<=cmd_yhi))[0]]
        
        ax.plot(x_iso,y_iso, zorder=300,color='red')

        ax.legend(['data', f'age = {age}'])

    x_l = np.linspace(cmd_xlo, cmd_xhi)    

    # Bin mid points
    y_rgbn = np.arange(l_lo, l_hi, dl)
    
    y_rgb_mid = y_rgbn[:-1] + dl/2   
    
    if fit_isochrone:
        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        model_iso = fit(init, y_i, x_i)
        
        ax.plot(model_iso(np.linspace(l_lo-0.1,l_hi+0.1)),
                          np.linspace(l_lo-0.1,l_hi+0.1),'--g', 
               zorder=400)
                
        if fit_slope:
            theta = np.arctan(-model_iso.slope.value)
        
    else:
        model_iso = interp1d(y_i, x_i)
        
    x_rgb_mid = model_iso(y_rgb_mid)
    
    ax.scatter(x_rgb_mid, y_rgb_mid, c='r' ,zorder = 200)

    dats = []
    
    dx = (cmd_xhi-cmd_xlo)/2
    for i,y0 in enumerate(y_rgbn[:-1]):

        # Extinction Vector
        x0 = model_iso(y0)
        y_Avl = y0 + np.tan(theta)*(x_l-x0)
        
        x01 = model_iso(y0 + dl)
        y_Avu = y0 + dl + np.tan(theta)*(x_l-x01)

        ax.plot(x_l,y_Avl, color='grey')
        ax.plot(x_l,y_Avu, color='grey')

        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        
        model_Avl = fit(init, x_l, y_Avl)
        model_Avu = fit(init, x_l, y_Avu)

        c1 = (y>model_Avl(x)) & (y<=model_Avu(x))
        c2 = (x>=x_rgb_mid[i]-dx) & (x<=x_rgb_mid[i]+dx) 

        yn = y[np.where(c1&c2)]
        xn = x[np.where(c1&c2)]
        
        dat = np.array([xn, yn])
        dats.append(dat)
    
    ax.set_xlabel(f"{filt1.upper()} - {filt2.upper()}")
    ax.set_ylabel(filt3.upper())

    ax.set_ylim(ylims[0], ylims[1])
    ax.set_xlim(xlims[0], xlims[1])  
    ax.invert_yaxis()
    ax.set_title(region.capitalize())

    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

    ax.yaxis.set_major_locator(AutoLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
    ax.tick_params(which='minor', length=4)
    
    AF1_ =  Av_dict[filt1]*Av_
    AF2_ =  Av_dict[filt2]*Av_
    AF3_ =  Av_dict[filt3]*Av_
    
    dx = AF1_ - AF2_
    dy = AF3_

    ax.annotate('', xy=(Av_x, Av_y),
                 xycoords='data',
                 xytext=(Av_x+dx, Av_y+dy),
                 textcoords='data',
                 arrowprops=dict(arrowstyle= '<|-',
                                 color='black',
                                 lw=0.5,
                                 ls='-')
               )

    ax.annotate(f'Av = {Av_}', xy=(Av_x-0.1, Av_y-0.1))
    
    return fig, ax, dats, x_rgb_mid, y_rgb_mid, y_rgbn

In [ ]:
def gen_CMD_ycut( filt1='f115w', filt2='f150w', filt3=None,region='bubble',name=None, Av=0.19, r=333,
                r_in=None, r_out=None, fig = None, ax = None, met=0.02, mag_err_lim=0.2, mag_err_cols = ['mag_err_1','mag_err_2'],
                dismod=29.95, cmd=None, label=3, ra_col = 'ra_1', dec_col= 'dec_1',
                out_dir='PHOT_OUTPUT_r25', xlims=[-0.5,2.5], ylims=[18,30], sqr_field=False,
                age=9.0, cmd_xlo = None, cmd_xhi= None, gen_kde=False, age_Z=False, perp_iso=False,
                l_lo = 22, l_hi=26.5, dl=0.5, Av_ = 3,ref_xpos=0.25,
                Av_x=2, Av_y=22, fit_isochrone=True,y0=None):
    
    if filt3 is None:
        filt3 = filt2
    if met is not None:
        if 'Zini' in cmd.keys():
            cmd = cmd[cmd['Zini']==met]
        else:
            cmd = cmd[cmd['Zini_1']==met]  
            
    cmd_ylo = l_lo
    cmd_yhi = l_hi
    
    age_lin = []
    for i in [age]:
        if i<6:
            age_lin.append(f'{np.ceil(10**i)} Myr')
        if i >= 6  and i <9:
            i-=6
            age_lin.append(f'{np.ceil(10**i)} Myr')
        elif i >= 9:
            i-=9
            age_lin.append(f'{np.ceil(10**i)} Gyr')
            
    
    if name is None:
        tab_bub = Table.read(f"../{out_dir}/{region}/{filt1}_{filt2}.fits")
    else:
        tab_bub = Table.read(f"../{out_dir}/{name}.fits")
        
    mag_errs = [i for i in tab_bub.keys() if 'mag_err' in i]
        
    for i in mag_errs:
        tab_bub = tab_bub[tab_bub[i]<=mag_err_lim]

    AF1 =  Av_dict[filt1]*Av
    AF2 =  Av_dict[filt2]*Av
    AF3 =  Av_dict[filt3]*Av

    ra_cen = regions_dict[region]['ra']
    dec_cen = regions_dict[region]['dec']

    tab_bub['r'] = angular_separation(tab_bub[ra_col]*u.deg,tab_bub[dec_col]*u.deg,
                                      ra_cen*u.deg, dec_cen*u.deg).to(u.arcsec).value
    
    ang = 245.00492
    ra_new = (tab_bub[ra_col]-ra_cen)*np.cos(np.pi/180*ang) - (tab_bub[dec_col]-dec_cen)*np.sin(np.pi/180*ang) + ra_cen
    dec_new = (tab_bub[ra_col]-ra_cen)*np.sin(np.pi/180*ang) + (tab_bub[dec_col]-dec_cen)*np.cos(np.pi/180*ang) + dec_cen

    tab_bub['temp_ra'] = tab_bub[ra_col].copy()
    tab_bub['temp_dec'] = tab_bub[dec_col].copy()
    tab_bub[ra_col] = ra_new
    tab_bub[dec_col] = dec_new
    
    if r_out is None or r_in is None :
        r_in = 0
        r_out = r

    if r_out is not None:    
        if not sqr_field:
            tab_bub = tab_bub[ (tab_bub['r']>=r_in) & (tab_bub['r'] < r_out)]
        else:
            
            tab_bub = tab_bub[ (tab_bub[ra_col]>=ra_cen-r_out/3600) & (tab_bub[ra_col]<=ra_cen+r_out/3600) ]
            tab_bub = tab_bub[ (tab_bub[dec_col]>=dec_cen-r_out/3600) & (tab_bub[dec_col]<=dec_cen+r_out/3600) ]

    tab_bub[ra_col] = tab_bub['temp_ra']
    tab_bub[dec_col] = tab_bub['temp_dec']
    tab_bub.remove_columns(['temp_ra', 'temp_dec'])
    
    x = tab_bub[f'mag_vega_{filt1.upper()}'] - tab_bub[f'mag_vega_{filt2.upper()}']
    y = tab_bub[f'mag_vega_{filt3.upper()}'] 
    
    x = x.value.astype(float)
    y = y.value.astype(float)
    
    if cmd_xlo is None or cmd_xhi is None:
        cmd_xlo = x.mean() - 1

        cmd_xhi = x.mean() + 1
    
    if fig is None or ax is None:
        fig = plt.figure(figsize=(7,7))
        ax = fig.add_subplot(1,1,1)
    l = []
    if gen_kde:

        # Peform the kernel density estimate
        xx, yy = np.mgrid[xlims[0]:xlims[1]:100j, ylims[0]:ylims[1]:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])
        values = np.vstack([x, y])

        kernel = gaussian_kde(values, bw_method=0.05)
        f = np.reshape(kernel(positions), xx.shape)

        f = f.T
        img = ax.imshow(f, cmap='jet', 
                      extent=[xlims[0], xlims[1], 
                              ylims[0], ylims[1]],
                       interpolation='nearest', aspect='auto')
            
    else:
        ax.scatter(x,y, s=0.2, color='black')
        l.append('data')
        
    l_ = [i for i in cmd.keys() if 'logAge' in i][0]
    
    if cmd is not None:
        t = cmd[np.round(cmd[l_],1)==age].copy()
        t = t[t['label']<=label]
        x_i =  (t[f'{filt1.upper()}mag'] + AF1) - (t[f'{filt2.upper()}mag'] + AF2)
        y_i =  t[f'{filt3.upper()}mag']
        # Max mag and Color
        M = y_i.min()
        c_ = x_i[y_i==y_i.min()].values[0]
        
        y_i +=   AF3 + dismod
        
        x_i = np.array(x_i)
        y_i = np.array(y_i)
        
        x_iso = x_i.copy()
        y_iso = y_i.copy()
        

        x_i = x_i[np.where( (y_i>=cmd_ylo-0.5) & (y_i<=cmd_yhi+0.5))[0]]
        y_i = y_i[np.where( (y_i>=cmd_ylo-0.5) & (y_i<=cmd_yhi+0.5))[0]]

        y_i = y_i[np.where( (x_i>=cmd_xlo-0.2) & (x_i<=cmd_xhi+0.2))[0]]
        x_i = x_i[np.where( (x_i>=cmd_xlo-0.2) & (x_i<=cmd_xhi+0.2))[0]]
        
        ax.plot(x_iso,y_iso, zorder=200, color='black')
        
        l.append(f'Age = {age_lin[0]}')

    x_l = np.linspace(0, 2)    

    # Bin mid points
    x_rgbn = np.arange(cmd_xlo, cmd_xhi, dl)
    
    x_rgb_mid = x_rgbn[:-1]+dl/2   
    
    if fit_isochrone:
        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        model_iso = fit(init, x_i, y_i)
        slope = 1/model_iso.slope.value
    
    elif perp_iso:
        slope=0
        
    else:
        model_iso = interp1d(x_i, y_i)
        slope = 1/model_iso.slope.value
    
    if y0 is None:
        y0 = y.mean()
    y_rgb_mid = y0 + x_rgb_mid*0

    dats = []
    
    x0 = 1# x.mean()
    init = models.Linear1D()
    fit = fitting.LinearLSQFitter()
    for x0 in x_rgbn[:-1]:
        
        y_l = np.linspace(cmd_ylo, cmd_yhi)
        x_l = slope*(y_l - y0) + x0
        
        y_r = np.linspace(cmd_ylo, cmd_yhi)
        x_r = slope*(y_r - y0) + x0 + dl

        ax.plot(x_l,y_l, color='red')
        ax.plot(x_r,y_r, color='red')

        init = models.Linear1D()
        fit = fitting.LinearLSQFitter()
        
        model_l = fit(init, y_l,x_l)
        model_r = fit(init, y_r,x_r)

        c1 = (x>model_l(y)) & (x<=model_r(y))

        yn = y[np.where(c1)]
        xn = x[np.where(c1)]
        if not gen_kde:
            ax.scatter(xn,yn, s =0.1, color='green', zorder=100)
        dat = np.array([xn, yn])
        dats.append(dat)
    
    ref = tab_bub[f'mag_vega_{filt2.upper()}']
    ref_new = np.arange(np.ceil(y.min()),np.ceil(y.max()) + 0.5,0.5)

    mag_err1 = tab_bub[mag_err_cols[0]]
    mag_err2 = tab_bub[mag_err_cols[1]]

    if len(mag_err_cols)>2:
        mag_err3 = tab_bub[mag_err_cols[2]]
    else:
        mag_err3 = mag_err2

    col_err = np.sqrt(mag_err1**2 + mag_err2**2)

    init = models.Exponential1D()
    fit = fitting.LevMarLSQFitter()
    model_col = fit(init,ref,col_err)

    init = models.Exponential1D()
    fit = fitting.LevMarLSQFitter()
    model_mag = fit(init,ref,mag_err3)

    x = ref_xpos + 0*ref_new
    y = ref_new
    yerr = model_mag(ref_new)
    xerr = model_col(ref_new)

    ax.errorbar(x, y, yerr,xerr ,fmt='o', color = 'red', markersize=0.5, capsize=2) 
    
    AF1_ =  Av_dict[filt1]*Av_
    AF2_ =  Av_dict[filt2]*Av_
    AF3_ =  Av_dict[filt3]*Av_
    
    dx = AF1_ - AF2_
    dy = AF3_

    ax.annotate('', xy=(Av_x, Av_y),
                 xycoords='data',
                 xytext=(Av_x+dx, Av_y+dy),
                 textcoords='data',
                 arrowprops=dict(arrowstyle= '<|-',
                                 color='black',
                                 lw=0.5,
                                 ls='-')
               )

    ax.annotate(f'Av = {Av_}', xy=(Av_x-0.1, Av_y-0.1))
    
    ax.set_xlabel(f"{filt1.upper()} - {filt2.upper()}")
    ax.set_ylabel(filt3.upper())

    ax.set_ylim(ylims[0], ylims[1])
    ax.set_xlim(xlims[0], xlims[1])  
    ax.invert_yaxis()
    title = f" | Z : {met} | " + "$M_" + "{" + f"{filt3.upper()}" +r"}^{TRGB}$ : " + f"{M} | "
    title += r"$A_{" + f"{filt3.upper()}" + r"}$ : " + f"{np.round(AF3,3)}"
    ax.set_title(region.capitalize() + title,
                fontsize=15)

    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

    ax.yaxis.set_major_locator(AutoLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
    ax.tick_params(which='minor', length=4)
    ax.legend(l)
    
    return fig, ax, dats, x_rgb_mid, y_rgb_mid, x_rgbn, [M, AF3, c_]

In [ ]:
Av_dict = { 
            'f275w': 2.02499,
            'f336w': 1.67536,
            'f435w': 1.33879,
            'f555w': 1.03065,
            'f814w': 0.59696,
            'f115w': 0.32726,
            'f150w': 0.21461,
            'f200w': 0.13589,
          }

In [ ]:
Av_dict = { 
            'f275w': 2.02499,
            'f336w': 1.67536,
            'f435w': 1.33879,
            'f555w': 1.03065,
            'f814w': 0.59696,
            'f115w': 0.419,
            'f150w': 0.287,
            'f200w': 0.195,
          }

In [ ]:
regions_dict = {'bubble' : {'ra'   : 24.1858128,  
                       'dec'  : 15.7725802,
                       'F115W': 25.4, 
                       'F150W': 24.63,
                       'F200W': 23.53,
                      
                       'F435W': 27.88, 
                       'F555W': 27.48,
                       'F814W': 26.52},
           
           'bkg1'   : {'ra'   : 24.1918038,
                       'dec'  : 15.7600032,
                       'F115W': 26., 
                       'F150W': 25.26,
                       'F200W': 24.21,
                      
                      'F435W': 28.26, 
                       'F555W': 27.94,
                       'F814W': 27.05},
              
           'bkg2'   : {'ra'   : 24.1787197,
                       'dec'  : 15.7543227,
                       'F115W': 26.31, 
                       'F150W': 25.39,
                       'F200W': 24.39,
                      
                      'F435W': 28.26, 
                       'F555W': 27.94,
                       'F814W': 27.05},
              
           'bkg3'   : {'ra'   : 24.1728133,
                       'dec'  : 15.7669357,
                       'F115W': 25.42, 
                       'F150W': 24.55,
                       'F200W': 23.56,
                      
                      'F435W': 27.88, 
                       'F555W': 27.48,
                       'F814W': 26.52},
                
            'galaxy' : {'ra'   : 24.1738983, 
                       'dec'  : 15.7836543,
                       'F115W': 25.42, 
                       'F150W': 24.55,
                       'F200W': 23.56},
                             
            'filament' : {'ra': 24.1498621,
                           'dec': 15.8016441,
                          },  
                
            'bubble_2' : {'ra': 24.1700350,
                           'dec': 15.8027816,
                          },
                
            'bubble_3' : {'ra':  24.1812907,
                           'dec': 15.7964848,
                          },
                 
            'bubble_4' : {'ra':  24.1802360,
                           'dec': 15.8082074,
                          },
                 
            'bubble_5' : {'ra':  24.1654172,
                           'dec': 15.7645942,
                          },  
            'bubble_6' : {'ra':  24.1783925,
                           'dec': 15.7485323,
                          }, 
                
            'outer' : {'ra':  24.1495874,
                           'dec': 15.8120031,
                          },
               }
with open('regions90.json') as json_file:
    data = json.load(json_file)
    
regions_dict.update(data)

## **JWST**

### **PARSEC Isochrones**

In [ ]:
with open("../data/isochrones_master/cmd_jwst_n.dat") as f:
    dat = f.readlines()

data = []

for i,d in enumerate(dat[13:]):
    if 'Zini' not in d and 'terminated' not in d:
        data.append([float(i) for i in d.split()])
        
df_cmd = pd.DataFrame(data,columns=dat[13][2:].split())
df_cmd.to_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv")

In [ ]:
ages= np.unique(df_cmd_jwst['logAge'])
mets = np.unique(df_cmd_jwst['Zini'])

In [ ]:
ages,mets

In [ ]:
if not os.path.exists('../data/isochrones/JWST'):
    os.mkdir('../data/isochrones/JWST/')
else:
    os.system('rm ../data/isochrones/JWST/*')


for i, age in enumerate([10]):  

    print(age)
    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)

    for met in mets:
        temp = df_cmd_jwst[np.round(df_cmd_jwst['logAge'],1)==age]
        temp = temp[(temp['Zini']==met) & (temp['label']<=10)]

        temp = temp[['label', 'Mini', 'F115Wmag', 'F150Wmag', 'F200Wmag', 'Zini', 'logAge']]

        header = list(temp.keys())
        header[0] = '#' + header[0]
        
#        if (met==0.02 and age<=9) or (met==0.006 and age>9):
        temp = temp[temp['label']<=3]
        temp.to_csv(f'../data/isochrones/JWST/{i}_PARSEC1.2S_Z{met}_logAGE{age}Myr_JWST_JHK.isoc',sep=' ',
                       index=None, header = header)

In [ ]:
ages_ = np.round(np.log10([6e6, 1e7,2e7, 3e7, 5e7,1e8,5e8, 1e9, 5e9, 1e10]),1)
mag_err_lim = 0.2

### **CMD**

Label: 0=PMS, 1=MS, 2=SGB, 3=RGB, (4,5,6)=different stages of CHEB, 7=EAGB, 8=TPAGB, 9=post-AGB

In [ ]:
r_in = 0
r_out = 10

fig,ax,tab = gen_CMD('f115w', 'f150w', cmd=df_cmd_jwst, r=None, gen_kde=False, 
                 r_in=r_in, r_out=r_out,Av_x=2, Av_y=27, Av_=2, ages=ages,
                 dismod=30.04,
                 label=3, met=0.006, out_dir='PHOT_OUTPUT_r25',  regions=['bubble'])

In [ ]:
r_in = 0
r_out = 10

fig,ax,tab = gen_CMD('f115w', 'f200w', cmd=df_cmd_jwst, r=None, xlims=[-0.5,1.8], Av_x=1,Av_y=27,
                  label=9, met=0.006, r_in=r_in, r_out=r_out, ages=ages,
                 out_dir='PHOT_OUTPUT_r25', regions=['bubble'])

In [ ]:
r_in = 0
r_out = 0.3

fig,ax1, tab = gen_CMD('f115w', 'f200w', 'f200w',name = 'stage2/f115w_f150w_f200w',
                       cmd=df_cmd_jwst[(df_cmd_jwst['Zini']>=0.003)&(df_cmd_jwst['Zini']<0.009)], r=None, Av_=1, Av_x=1.5, Av_y=27, xlims=[-0.5,2], ylims=[17,30],
                 out_dir='PHOT_OUTPUT_r40.5',gen_kde=False, label_min=0, label_max=3, met=None,dismod=30.04, ages=[9.0,9.2,9.4,9.6,9.8,10.,10.1],
                 r_in=r_in, r_out=r_out,regions=['bubble'])
ax1.set_title('')
#ax1.get_legend().remove() 

In [ ]:
r_in = 0
r_out = 12
fig,ax1, tab = gen_CMD('f115w', 'f200w', 'f115w',name = 'ngc628/f115w_f200w',  cmd=None, r=None, Av_=1, Av_x=2.5, Av_y=26, xlims=[-0.5,4], ylims=[17,28],
                 out_dir='photometry',gen_kde=False, label=10, met=0.003,dismod=29.95, ages=ages_,
                 r_in=r_in, r_out=r_out,
                regions=['bkg3'])

In [ ]:
x = tab['mag_vega_F200W']
y = tab['mag_err_2']

fig, ax = plt.subplots()

ax.scatter(x,y,s=0.1, color='black')

ax.set_xlabel('F200W (vega)')
ax.set_ylabel('mag_err')

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
ax.tick_params(which='minor', length=4)

In [ ]:
r_in = 0
r_out = 10

fig, ax = plt.subplots(1,2, sharey=False,figsize=(22,10))


fig,ax1, tab = gen_CMD('f115w', 'f200w', cmd=df_cmd_jwst, r=None, Av_=1, Av_x=2.5, Av_y=26, xlims=[-0.5,4], ylims=[17,28],
                 out_dir='PHOT_OUTPUT_r25',gen_kde=False, label=10, met=0.006,dismod=30.04, ages=ages,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[0],
                regions=['bubble'])

fig,ax2, tab = gen_CMD('f115w', 'f200w', cmd=df_cmd_jwst, r=None, Av_=1, Av_x=2.5, Av_y=26, xlims=[-0.5,4], ylims=[17,28],
                 out_dir='PHOT_OUTPUT_r25',gen_kde=True, label=3, met=0.006,dismod=30.04, ages=ages,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[1],
                regions=['bubble'])

In [ ]:
r_in = 0
r_out = 12

fig,ax, tab1 = gen_CMD('f115w', 'f200w', 'f115w', name = 'ngc628n/f115w_f200w', cmd=None, Av_=1, Av_x=2.5, Av_y=26, xlims=[-1,3], ylims=[17,28],
                 out_dir='photometry',gen_kde=False,gen_contours=False, label=10, met=0.02,mag_err_lim=0.2, alpha=0.5,
                dismod=29.95, ages=ages_, sqr_field=True, r_in=r_in, r_out=r_out, regions=['bubble'])

In [ ]:
x = tab1['ra_1']
y = tab1['dec_1']
c = tab1['mag_vega_F200W']
fig, ax = plt.subplots()

ra_cen = regions_dict['bubble']['ra']
dec_cen = regions_dict['bubble']['dec']

ax.scatter([ra_cen],[dec_cen], marker='x', color='black')

ax.invert_xaxis()
ax.scatter(x,y,c=c,s=5, cmap='jet')
ax.axis('equal')

In [ ]:
r_in = 0
r_out = 12

fig, ax = plt.subplots(1,3, sharey=False,figsize=(34,10))

filt1 = 'f115w'

filt2 = 'f200w'

filt3 = 'f115w'

fig,ax1, tab = gen_CMD(filt1, filt2,filt3, name = 'ngc628/f115w_f200w',
                       cmd=None, Av_=1, Av_x=1.5, Av_y=18, xlims=[-1,5], ylims=[17,28],
                     out_dir='photometry',gen_kde=False,gen_contours=True, label=10, met=0.02,
                    dismod=30.04, ages=ages_, sqr_field=False, r_in=r_in, r_out=r_out, regions=['galaxy'],
                    fig=fig,ax=ax[0],ref_xpos=-0.5)

fig,ax2, tab = gen_CMD(filt1, filt2,filt3, name = 'ngc628/f115w_f200w',
                       cmd=df_cmd_jwst, r=None, Av_=1, Av_x=1.5, Av_y=18, xlims=[-1,4], ylims=[17,28],
                 out_dir='photometry',gen_kde=False, label=3, met=0.003,dismod=29.95, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[1],ref_xpos=-0.5,
                regions=['bubble'])

fig,ax3, tab = gen_CMD(filt1, filt2, filt3,name = 'ngc628/f115w_f200w',
                       cmd=df_cmd_jwst, r=None, Av_=1, Av_x=1.5, Av_y=18, xlims=[-1,4], ylims=[17,28],
                 out_dir='photometry',gen_kde=False, label=3, met=0.02,dismod=29.95, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[2],ref_xpos=-0.5,
                regions=['bubble'])

In [ ]:
#ages_ = np.round(np.log10([6e6, 1e7,2e7, 3e7, 5e7,1e8,5e8]),1)
ages_ = np.round(np.log10([1e9, 5e9, 1e10]),1)

In [ ]:
r_in = 0
r_out = 10

fig, ax = plt.subplots(1,4, sharey=False,figsize=(44,10))

fig,ax1, tab = gen_CMD('f115w', 'f150w', 'f150w', name = '3004/f115w_f150w_f200w',
                       cmd=df_cmd_jwst, Av_=1, Av_x=-2.5, Av_y=18, xlims=[-0.5,1.5], ylims=[17,28],
                     out_dir='PHOT_OUTPUT_r40.5',gen_kde=True,gen_contours=False, label=3, met=0.003,
                    dismod=30.04, ages=ages_, sqr_field=False, r_in=r_in, r_out=r_out, 
                       regions=['bubble'],
                    fig=fig,ax=ax[0],ref_xpos=-0.5)

fig,ax2, tab = gen_CMD('f150w', 'f200w', 'f200w', name = '3004/f115w_f150w_f200w',
                       cmd=df_cmd_jwst, r=None, Av_=-2.5, Av_x=1.5, Av_y=18, xlims=[-0.5,1.5], ylims=[17,28],
                 out_dir='PHOT_OUTPUT_r40.5',gen_kde=True, label=3, met=0.003,dismod=30.04, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[1],ref_xpos=-0.5,
                regions=['bubble'])

fig,ax3, tab = gen_CMD('f115w', 'f200w', 'f200w',name = '3004/f115w_f150w_f200w',
                       cmd=df_cmd_jwst, r=None, Av_=1, Av_x=1.5, Av_y=18, xlims=[0.5,2.5], ylims=[20,28],
                 out_dir='PHOT_OUTPUT_r40.5',gen_kde=True, label=3, met=0.003,dismod=30.04, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[2],ref_xpos=-0.5,
                regions=['bubble'])

fig,ax4, tab = gen_CMD('f115w', 'f200w', 'f115w',name = '3004/f115w_f150w_f200w',
                       cmd=df_cmd_jwst, r=None, Av_=1, Av_x=1.5, Av_y=18, xlims=[0.5,2.5], ylims=[20,28],
                 out_dir='PHOT_OUTPUT_r40.5',gen_kde=True, label=3, met=0.003,dismod=30.04, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[3],ref_xpos=-0.5,
                regions=['bubble'])

In [ ]:
r_in = 0
r_out = 12

fig, ax = plt.subplots(6,15,figsize=(120,60), sharex=True, sharey=True)

k=0
for i in range(15):
    for j in range(6):
        fig,ax1, _ = gen_CMD('f115w', 'f200w', name = 'ngc628/f115w_f200w', cmd=None, r=None, Av_=1, Av_x=2.5, Av_y=26, xlims=[-0.5,4], ylims=[17,28],
                         out_dir='photometry',gen_kde=False, label=10, met=0.006,dismod=30.04, ages=ages_, sqr_field=True,
                         r_in=r_in, r_out=r_out, fig=fig,ax=ax[5-j,i], ab_dist=False,
                         regions=[f'reg_{k}'])
        if j!=0:
            ax1.set_xlabel(None)
        if i!=0:
            ax1.set_ylabel(None)
        k+=1

### **Extinction Vector**

In [ ]:
23.4+0.025

In [ ]:
filt1 = 'f115w'
filt2 = 'f200w'
r_in  = 0
r_out = 10

dl= 0.05

l_lo = 23 
l_hi = 23.5
cmd_xlo = 0.5
cmd_xhi = 2
fig, ax, dats_bub, x_val, y_val, y_bins = gen_CMD_xcut(filt1, filt2,  name = 'stage2/f115w_f150w_f200w',
                                                       cmd=df_cmd_jwst, label_min=0, label_max=3,
                                                      r_in=r_in, r_out=r_out, age=10, met=0.006,
                                                      l_lo=l_lo, l_hi=l_hi, out_dir='PHOT_OUTPUT_r40.5',
                                                      dl=dl,cmd_xlo=cmd_xlo, cmd_xhi=cmd_xhi,
                                                       region='bubble',
                                                     fit_isochrone=True)


fig, ax, dats_bkg3, x_val, y_val, y_bins = gen_CMD_xcut(filt1, filt2,  name = 'stage2/f115w_f150w_f200w',
                                                        cmd=df_cmd_jwst,label_min=0, label_max=3,
                                                       r_in=r_in, r_out=r_out, age=10, met=0.006,
                                                       l_lo=l_lo, l_hi=l_hi,out_dir='PHOT_OUTPUT_r40.5',
                                                      dl=dl,cmd_xlo=cmd_xlo, cmd_xhi=cmd_xhi,
                                                         region='bkg3',fit_isochrone=True)

In [ ]:
min_ = 1
max_ = 2
step = 0.025
bins = np.arange(min_, max_, step)

init = models.Gaussian1D()
fit = fitting.LevMarLSQFitter()

for i in range(len(y_val)):
    
    fig, ax = plt.subplots()
    dat1 = dats_bub[i][0]
    dat2 = dats_bkg3[i][0]

   # Bubble
    y, x = np.histogram(dat1, bins=bins)
    max_bub = y.max()
    
    x = 0.5*(x[1:] + x[:-1])
    
    ax.step(x,y, where='mid', color='black')
    
    model = fit(init,x,y)
    xn = np.linspace(x.min(),x.max(),1000)
    
    ax.plot(xn,model(xn),'-b')
    
    x_p1 = model.mean.value
    s_p1 = model.stddev.value
    ax.plot([x_p1, x_p1],[0,model.amplitude.value], '--b')
    
    # Bkg3
    y, x = np.histogram(dat2, bins=bins)
    
    max_bkg3 = y.max()
    
    x = 0.5*(x[1:] + x[:-1])
    
    ax.step(x,y, where='mid', color='gold')
    
    model = fit(init,x,y)
    xn = np.linspace(x.min(),x.max(),1000)
    ax.plot(xn,model(xn),'-', color='orange')
    
    x_p3 = model.mean.value
    s_p3 = model.stddev.value
    ax.plot([x_p3, x_p3],[0,model.amplitude.value], '--', color='orange')
    
    max_ = np.max([max_bub,  max_bkg3])
    
    
    ax.plot([x_val[i], x_val[i]], [0, max_+10],'-g',linewidth=3)
    title = f"{np.round(y_bins[i],2)} - {np.round(y_bins[i+1],3)} (mag)| "
    title += r"$\mu_{bubble} :$" + f"{np.round(x_p1,3)}" + r" $\mu_{bkg3} :$" + f"{np.round(x_p3,3)} " 
    
    title += r'$\sigma_{bubble} : $ ' + f"{np.round(s_p1,3)} " + r' $\sigma_{bkg3} : $ ' + f"{np.round(s_p3,3)}\n"
    title +=  r"$\ dist_{bub,ref}$ : " + f"{np.round(x_p1 - x_val[i],3)}  " 
    title +=   r"$\ dist_{bkg3,ref}$ : " + f"{np.round(x_p3 - x_val[i],3)}"
    
    ax.set_title(title, fontsize=15)
    ax.set_ylabel('Counts', fontsize=15)
    ax.legend(['Bubble', 'Bubble Gaussian Fit', 'Bubble mean', 'Bkg3', 'Bkg3 Gaussian Fit', 'Bkg3 mean',
                 'Reference line'])
    
    ax.set_xlabel('Color');
    plt.show()

## **TRGB**

### **Magnitude**

In [ ]:
dismod = 30.04

In [ ]:
filt1 = 'f115w'
filt2 = 'f200w'
filt3 = 'f115w'

r_in  = 0
r_out = 12

if filt3 == 'f200w':
    dl= 0.25
elif filt3 == 'f115w':
    dl = 0.25
elif filt3== 'f150w':
    dl = 0.25
    
l_lo = 19
l_hi = 28

if filt3 == 'f200w':
    cmd_xlo = 1.25
    y0 = 23.5
elif filt3 == 'f115w':
    cmd_xlo = 1.25
    y0 = 24.8
elif filt3 == 'f150w':
    cmd_xlo = 1.23
    y0 = 24.

cmd_xhi = cmd_xlo + dl*1.1

nx = 15
ny = 6

trgb_mags = np.zeros((nx, ny, 6))*np.nan

if filt3 == 'f200w':
    col_lims = [23.20, 23.7]
    
elif filt3 == 'f115w':
    col_lims = [24.4, 25.1]
    
elif filt3 == 'f150w':
    col_lims = [23.5, 24.3]

k = 72

for i in range(nx):
    for j in range(ny):
        mss = []
        for b in np.arange(0.001,0.011,0.001):
            bins = np.arange(l_lo, l_hi, b)
            ms = []
            for sigma_s in np.arange(0.05,0.11,0.01):

                fig = plt.figure(figsize=(14, 7))
                gs = fig.add_gridspec(1, 3,  width_ratios=(4, 1,1),
                                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                                      wspace=0.05, hspace=0.05)

                ax =  fig.add_subplot(gs[0, 0])

                fig, ax, dats_bub, x_val, y_val, y_bins,params = gen_CMD_ycut(filt1, filt2, filt3, name= 'ngc628/f115w_f200w',
                                                                      region=f'reg_{k}',cmd=df_cmd_jwst, 
                                                                      r_in=r_in, r_out=r_out, label=3,
                                                                      met=0.003, sqr_field=False, mag_err_lim=0.2,
                                                                      age=10, fig=fig, ax=ax, gen_kde=True,
                                                                      l_lo=l_lo, l_hi=l_hi, out_dir='photometry',
                                                                      dl=dl, cmd_xlo=cmd_xlo, cmd_xhi=cmd_xhi,
                                                                       xlims=[0.,3], ylims=[21,27], y0=y0,
                                                                      fit_isochrone=True, perp_iso=True, dismod=29.83)

                ax_hist = fig.add_subplot(gs[0, 1], sharey=ax)
                ax_edge = fig.add_subplot(gs[0, 2], sharey=ax)


                for dat in dats_bub:
                    x = dat[0]
                    y = dat[1]

                    yh,xh,_, = ax_hist.hist(y,bins, orientation='horizontal' , histtype='step', color='grey')
                    xh = 0.5*(xh[1:] + xh[:-1])
                    #yf = gaussian_filter(yh,5) 

                    yf = gloess(xh,yh,sigma_s,1)
                    ysob = np.convolve(yf,np.array([-1,0,1]), mode='same')

                    ax_hist.plot(yf, xh, color='black')
                    ax_edge.plot(ysob, xh,'b')
                    c1 = (xh>=col_lims[0]) & (xh<=col_lims[1])
                    m = xh[c1][ysob[c1]==ysob[c1].min()][0]

                    ax_hist.tick_params(axis="y", labelleft=False)
                    ax_edge.tick_params(axis="x", labelbottom=False)
                    ax_edge.tick_params(axis="y", labelleft=False)

                    ax_edge.set_title(f'{filt3.upper()}'+ '$_{TRGB}$' f': {np.round(m,5)}', fontsize=12)
                    ax.plot([0,3],[m,m],'--k', zorder=201)
                    ax_hist.plot([0,yh.max()],[m,m],'--k',label=f'{filt3.upper()}'+ '$_{TRGB}$')
                    ax_hist.legend()
                    ax_hist.set_xticks([np.ceil(yh.max())],[np.ceil(yh.max())])
                    ax_hist.tick_params(axis='both', labelsize=15)
                    dismod = m - params[0]-params[1]
                    ax_hist.set_title(f'DM : {np.round(dismod, 3)}', fontsize=15)
                    ax_edge.plot([ysob.min(),ysob.max()],[m,m],'--k')
                    ax_edge.set_xlim([ysob.min()*1.1,0])
                    #ax_edge.invert_xaxis()

                    for ax_ in [ax_hist, ax_edge]:
                        ax_.xaxis.set_major_locator(AutoLocator())
                        ax_.xaxis.set_minor_locator(AutoMinorLocator())

                        ax_.yaxis.set_major_locator(AutoLocator())
                        ax_.yaxis.set_minor_locator(AutoMinorLocator())

                        ax_.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
                        ax_.tick_params(which='minor', length=4)

                    if m >= col_lims[0] and m <= col_lims[1] and ysob[c1].min() < 0:
                        trgb_mags[i,j,0] = m
                        trgb_mags[i,j,1]= len(y[(y>m) & (y<m+0.1)]) # TRGB Window
                        trgb_mags[i,j,2]= len(y[(y>m) & (y<m+1)]) # RGB stars
                        trgb_mags[i,j,3]= len(y[(y<m)]) # AGB stars
                        
                    
                    fig.savefig(f"TRGB/{filt3}/reg_{filt3}_{k}_{sigma_s}_{b}_mag.png")
                    plt.close(fig) 
                            
                ms.append(m)
            mss.append(ms)

        print(k)
        
        k+=1
        m_t= np.array(mss)
        trgb_mags[i,j,4] = np.median(m_t)
        trgb_mags[i,j,5] = m_t.std()
        break
    break
    
#np.save(f'TRGB/trgb_mags_{filt3}.npy', trgb_mags)

In [ ]:
m_t = np.array(mss)
np.nanmedian(m_t), np.nanstd(m_t), np.nanmedian(m_t) - params[0] - params[1]

In [ ]:
bins = np.arange(0.001, 0.011, 0.001)

for n,i in enumerate(m_t.T):
    plt.plot(bins, i, '-o')
    plt.legend(np.arange(0.05, 0.11, 0.01))

In [ ]:
bubble_f115w        = 24.785
bubble_f115w_dismod = 29.81439000000091
bubble_f115w_err    = 0.0062

trgb_f115w_mag = np.array([[24.8545, 24.8557, 24.9077, 25.0250, 24.903, 24.86749],
                           [24.8329, 24.9319, 24.9220, 24.8900, 24.83475, 24.8749],
                           [24.8529, 24.94475, np.nan, 24.9198, 24.8465,  24.851999],
                           [ 24.84850,  24.8905,  24.84575,  24.914249,  25.015,  np.nan],
                           [ 24.947999,  24.89425,  24.860749,  24.819999,  24.804750,  24.866749],
                           [ 24.893249,  24.835250,  24.7930,  24.8485,  24.82750,  24.697249],
                           [ 24.8445,  24.885250,  np.nan,  np.nan,  np.nan,  np.nan],
                           [ np.nan,  np.nan,  np.nan,  np.nan,  np.nan,  np.nan],
                           [ 24.79950,  np.nan,  np.nan,  np.nan,  24.79375,  24.799500],
                           [ 24.782749,  24.8774999,  24.8905,  24.75100,  24.87075,  24.9100],
                           [ 24.947749,  24.8895,  24.862500,  24.89050,  24.86950,  24.85749],
                           [ 24.83325,  24.890250,  24.835750,  24.80399,  24.82550,  24.8974999],
                           [ 24.83650,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True]])

trgb_f115w_err = np.array([[0.0037,  0.0080,  0.0081,  0.0083, 0.0084, 0.0094],
                           [0.0106,  0.0114,  0.00695, 0.0149, 0.0117, 0.01469],
                           [0.0018,  0.01113, np.nan,  0.0156, 0.0031,  0.00652],
                           [0.00266, 0.00355, 0.00126,  0.005261,  0.009413,  np.nan],
                           [0.0083595,  0.002936,  0.00395,   0.00794,  0.004538,  0.00480],
                           [0.007299,  0.00282690,  0.0064,  0.002324,  0.00114,  0.00606],
                           [0.00533,  0.001123,  np.nan, np.nan,  np.nan,  np.nan],
                           [np.nan,  np.nan,  np.nan,  np.nan,  np.nan,  np.nan],
                           [ 0.0023,   np.nan,  np.nan,  np.nan,  0.01046,  0.00276],
                           [ 0.0031679,  0.002494,  0.001929,  0.00310,  0.0160412,  0.020460],
                           [ 0.0051674,  0.0062207,  0.0079311,  0.00734773,  0.007624,  0.00735921],
                           [ 0.0085020,  0.0031088,  0.0057223,  0.00722214,  0.00694712,  0.0034528],
                           [ 0.006362,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True]])

In [ ]:
bubble_f200w         = 23.36699999999758
bubble_f200w_dismod  = 29.80094999999758
bubble_f200w_err     = 0.00514

trgb_f200w_mag = np.array([[23.48575, 23.465, 23.525, 23.56575, 23.53075, 23.4455],
                           [23.47325, 23.5937, 23.53775, 23.5065, 23.4595, 23.4955],
                           [23.49150, 23.47599, np.nan, 23.540249,  23.44150,  23.4625],
                           [23.46975, 23.3575,  23.49925,  23.3965,  23.58700,  np.nan],
                           [23.45500,  23.4599,  23.46250,  23.429499,  23.417249,  23.39850],
                           [23.38749,  23.4650,  23.40100,  23.347999,  23.34799,  23.15350],
                           [ 23.38300,  23.4332500,  np.nan,  np.nan,  np.nan, np.nan],
                           [ np.nan,  np.nan,  np.nan,  np.nan,  np.nan,  np.nan],
                           [ 23.36924,  np.nan,  np.nan,  np.nan,  23.26950, 23.33650],
                           [ 23.3065,  23.534250,  23.448750,  23.35150,  23.40875,  23.394499],
                           [ 23.52850,  23.47500,  23.468500,  23.39599,  23.425000,  23.46250],
                           [ 23.46250,  23.46824999,  23.49050,  23.3967499,  23.45350,  23.4985000],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True]])

trgb_f200w_err = np.array([[0.0125,  0.0067,  0.0054,  0.0084,  0.0052,  0.0089],
                           [0.0035,  0.01058,  0.00536, 0.0154, 0.0044,  0.0043],
                           [0.0054,   0.0039,  np.nan,  0.0046, 0.00475,  0.005595],
                           [0.00513,  0.00212,  0.00270,  0.00758,  0.009482,  np.nan],
                           [ 0.005,  0.005885,  0.00442,  0.011658926,  0.00768,  0.00431],
                           [ 0.006399,  0.00402,  0.01538,  0.007429,  0.003502,  0.002646],
                           [ 0.0032626,  0.008600,  np.nan,  np.nan,  np.nan, np.nan],
                           [ np.nan,  np.nan,  np.nan,  np.nan,  np.nan,  np.nan],
                           [ 0.0028210, np.nan,  np.nan,  np.nan, 0.0085117, 0.008598],
                           [ 0.013819,  0.006270,   0.001844,  0.00821,  0.0141245,  0.00789011],
                           [ 0.005601,  0.005485,  0.005807,  0.00734773,  0.00714082, 0.0028327],
                           [ 0.003719,  0.0094440,  0.00442574,  0.005583,  0.001789553,  0.00652],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True],
                           [ True,  True,  True,  True,  True,  True]])

In [ ]:
trgb_mags_J = trgb_f115w_mag#np.load('TRGB/trgb_mags_f115w.npy')
trgb_mags_K = trgb_f200w_mag#np.load('TRGB/trgb_mags_f200w.npy')

In [ ]:
mags_f115w = []
mags_f200w = []

ages__ = [9.4, 9.6,9.8,10.00001]
for age in ages__:
    mags1 = []
    mags2 = []
    for met in [0.002,0.003,0.004, 0.006,0.008]:
        df = df_cmd_jwst[ (df_cmd_jwst['logAge']==age) & (df_cmd_jwst['Zini']==met) & (df_cmd_jwst['label']<=3)]

        t = df[df['F115Wmag']==df['F115Wmag'].min()]
        mag = t['F115Wmag'].values[0]       
        mags1.append(mag)
        
        t = df[df['F200Wmag']==df['F200Wmag'].min()]
        mag = t['F200Wmag'].values[0]       
        mags2.append(mag)
        
    mags_f115w.append(mags1)
    mags_f200w.append(mags2)

mags_f115w = np.array(mags_f115w)
mags_f200w = np.array(mags_f200w)

In [ ]:
yy, xx  =np.mgrid[0:5,0:15]

In [ ]:
z = (xx-7)**2 + (yy-2)**2

In [ ]:
nx = 15
ny = 6

k=0
reg_arr = np.zeros_like(trgb_mags_J[:,:])
for i in range(nx):
    for j in range(ny):
        reg_arr[i,j]=k
        k+=1

In [ ]:
Av = 0.19
y_r = mags_f200w[-1] + Av_dict['f200w']*Av
x_r = mags_f115w[-1] + Av_dict['f115w']*Av - (mags_f200w[-1] +  Av_dict['f200w']*Av)

x_10 = np.arange(1.35,1.5,0.001)
y_10 = np.interp(x_10,x_r,y_r)
met_10 = np.interp(x_10,x_r, [0.002,0.003,0.004, 0.006,0.008])
x_c = bubble_f115w-bubble_f200w
y_c = bubble_f200w  

d = np.arange(29,30,0.0001).reshape(-1,1)
    
r = np.sqrt( (x_10 - x_c)**2 + (y_10 + d - y_c)**2)

dismod= d[np.argwhere(r==r.min())[0][0]][0]
met_10[np.argwhere(r==r.min())[0][1]]

In [ ]:
print(dismod)

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))

dismod = 29.831499
Av = 0.19

Av_t = 0.0

x = (mags_f115w.T + Av_dict['f115w']*(Av+Av_t)) - (mags_f200w.T + Av_dict['f200w']*(Av+Av_t))
y = mags_f200w.T + Av_dict['f200w']*(Av+Av_t) + dismod

ax.plot(x,y, '-o',
       linewidth=0.5, markersize=1)

start = 50
end = 71
    
x = trgb_mags_J[:,:].ravel()[start:end+1] - trgb_mags_K[:,:].ravel()[start:end+1]
y = trgb_mags_K[:,:].ravel()[start:end+1]
    
xerr = trgb_f115w_err[:,:].ravel()[start:end+1]
yerr = trgb_f200w_err[:,:].ravel()[start:end+1]

xerr = np.sqrt(xerr**2 + yerr**2)

c = z.ravel()*12
    
img = ax.errorbar(x,y,xerr=xerr, yerr=yerr, fmt='.k',elinewidth=0.5,
            markersize=0.5, capsize=2, zorder=100)

ax.invert_yaxis()
#ax.invert_xaxis()

Av_x = 24.781
Av_y = 23.349 

Av_x = Av_x-Av_y

Av_ = 0.3

AF1_ =  Av_dict['f115w']*Av_
AF2_ =  Av_dict['f200w']*Av_

dx = AF1_
dy = AF2_

dx = dx-dy
ax.annotate('', xy=(Av_x, Av_y),
             xycoords='data',
             xytext=(Av_x+dx, Av_y+dy),
             textcoords='data',
             arrowprops=dict(arrowstyle= '<|-',
                             color='black',
                             lw=0.5,
                             ls='-')
           )

for n,i,j in zip(reg_arr.ravel().astype(int)[start:end+1],x,y):
    ax.annotate(f'{n}', xy=(i- 0.001, j-0.001),
             xycoords='data', fontsize=7, color = 'red', zorder= 100, weight='bold'
           )

x = bubble_f115w-bubble_f200w
y = bubble_f200w

xerr = np.sqrt(bubble_f115w_err**2 + bubble_f200w_err**2)
yerr = bubble_f200w_err
ax.errorbar([x],[y],xerr=xerr,yerr=yerr, fmt='.r',elinewidth=0.5,
            markersize=0.5, capsize=2)

Av = 0.19
r_ind = -1
y_r = mags_f200w[r_ind] + Av_dict['f200w']*Av + dismod
x_r = mags_f115w[r_ind] + Av_dict['f115w']*Av - (mags_f200w[r_ind] +  Av_dict['f200w']*Av)

ax.plot(x_r, y_r,'--k')
ax.set_xlabel('F115W-F200W')
ax.set_ylabel('F200W')
ax.set_title(f'Dismod = {np.round(dismod,4)}')
ax.legend(ages__)

In [ ]:
Av = 0.19
dismod = 29.831499
ind = -1
y_r = mags_f200w[ind] + Av_dict['f200w']*Av + dismod
x_r = mags_f115w[ind] + Av_dict['f115w']*Av - (mags_f200w[ind] +  Av_dict['f200w']*Av)

x_10 = np.arange(1.35,1.5,0.001)
y_10 = np.interp(x_10,x_r,y_r)
met_10 = np.interp(x_10,x_r, [0.002,0.003,0.004, 0.006,0.008])

ind = 71

x_c = trgb_mags_J[:,:].ravel()[ind] - trgb_mags_K[:,:].ravel()[ind]
y_c = trgb_mags_K[:,:].ravel()[ind]

Av_ = np.arange(0,0.5, 0.001).reshape(-1,1)

dx = (Av_dict['f115w']-Av_dict['f200w'])*Av_
dy = Av_dict['f200w']*Av_
    
r = np.sqrt( (x_10 - (x_c-dx))**2 + (y_10 - (y_c-dy))**2)

Av_[np.argwhere(r==r.min())[0][0]][0], met_10[np.argwhere(r==r.min())[0][1]]

In [ ]:
Av_map = np.array([[0.033, 0.067, 0.139, 0.44, 0.123, 0.131],
                   [0, np.nan, 0.169, 0.111, 0.011, 0.082],
                   [0.02, 0.322,  np.nan,  0.163,  0.073,  0.059000],
                   [ 0.038,  0.364,  0.0,  0.363,  0.389,  np.nan],
                   [ 0.368,  0.191,  0.0860,  0.01,  0.0,  0.2070],
                   [ 0.313,  0.004,  0.0,  0.242,  0.171,  0.14300],
                   [ 0.163,  0.203,  np.nan,  np.nan,  np.nan,  np.nan],
                   [ np.nan,  np.nan,  np.nan,  np.nan,  np.nan, np.nan],
                   [ 0.111,  0.044,  0.057,  0.0,  0.201,  0.353],
                   [ 0.257,  0.156,  0.082,  0.2880,  0.168,  0.076],
                   [ 0.001,  0.167,  0.0,  0.00900,  0.0,  0.146],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True]])

met_map = np.array([[0.002297, 0.002539, 0.0021361, 0.002297, 0.0021200, 0.002813],
                   [0.002329, np.nan,   0.0021200, 0.002249, 0.0024909, 0.002297],
                   [0.002232, 0.002861,  np.nan,  0.00212,  0.002749,  0.0025393],
                   [0.0024425,  0.004419,  0.002120,  0.003849, 0.00212,  np.nan],
                   [0.00317,  0.002781,  0.0025877,  0.0027490,  0.00281354,  0.0034713],
                   [0.0038497,  0.00242,  0.002926,  0.0041973,  0.0039848,  0.0070366],
                   [0.00355243,  0.0030389,  np.nan,  np.nan,  np.nan,  np.nan],
                   [np.nan,  np.nan,  np.nan,  np.nan,  np.nan, np.nan],
                   [0.0044639,  0.00540,  0.002120,  0.005242,  0.0033362,  0.003849],
                   [ 0.0023135,  0.00260387,  0.00252,  0.003687567,  0.003065,  0.002571],
                   [ 0.0024425,  0.002668,  0.00216838,  0.00303,  0.00250,  0.00237806],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True],
                   [ True,  True,  True,  True,  True,  True]])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10))

img = ax[0].imshow(Av_map.T, cmap='jet', vmin=0, vmax= 0.4)
plt.colorbar(img,ax=ax[0], orientation='horizontal')
img= ax[1].imshow(met_map.T, cmap='jet_r', vmin=0.002, vmax= 0.007)
plt.colorbar(img,ax=ax[1], orientation='horizontal')

### **TRGB Color**

In [ ]:
filt1 = 'f115w'
filt2 = 'f200w'
filt3 = 'f200w'

dl= 0.05
cmd_xlo = 1.20
cmd_xhi = 1.6

r_in  = 0
r_out =  10#29.987035918392053/2
dats = []
nx = 9
ny = 4
trgb_cols = np.zeros((nx,ny,7))*np.nan
k = 0

min_ = cmd_xlo
max_ = cmd_xhi
step = 0.01

bins = np.arange(min_, max_, step)
init = models.Gaussian1D()
fit = fitting.LevMarLSQFitter()

for i in range(nx):
    for j in range(ny):
        m = trgb_mags[i,j,0]
        
        if np.isnan(m):
            print(k)
            continue
            
        l_lo = m - 0.01
        l_hi = m + 0.05
        dats = []
        fig, ax = plt.subplots(1,2,figsize=(16, 8))
        _, _, dat, x_val, y_val, y_bins = gen_CMD_xcut(filt1, filt2, filt3,name= 'stage2/f115w_f200w', cmd=df_cmd_jwst,
                                                           label_min=2, label_max=3, Av_=1, Av_x=1.5,Av_y=23.5,
                                                           r_in=r_in, r_out=r_out, age=10., met=0.003, sqr_field=True,
                                                           l_lo=l_lo, l_hi=l_hi, out_dir='PHOT_OUTPUT_r40.5',
                                                           dl=dl, cmd_xlo=cmd_xlo, cmd_xhi=cmd_xhi,
                                                           cmd_ylo=23, cmd_yhi=25,
                                                           fit_slope=True,
                                                           region=f'bubble',xlims=[0,3],ylims=[22, 26],
                                                           fit_isochrone=True, dismod=29.718,
                                                           fig=fig, ax=ax[0])
        #plt.close(fig)
        dats.append(dat)


        dat = dats[0][0][0]

        y, x = np.histogram(dat, bins=bins)
        
        max_bub = y.max()

        x = 0.5*(x[1:] + x[:-1])
        
        ax[1].step(x,y, where='mid', color='black')
        
        model = fit(init, x,y)
        ax[1].plot(np.linspace(min_, max_,100), model(np.linspace(min_, max_,100)),'-r')
       
        mean, median, stddev= sigma_clipped_stats(dat, sigma=2, cenfunc=biweight_location, stdfunc=biweight_scale, maxiters=10)

        p = x[y==y.max()][0]
        x_p1 = mean
        s_p1 = stddev

        ax[1].plot([x_p1, x_p1],[0,y.max()], '--b')

        title = ''
        title += r"$Biweighted_{\mu}:$ " + f"{np.round(x_p1,4)} | "
        title += r'$Biweighted_{\sigma}:$ ' + f"{np.round(s_p1,4)}"
        title += r' | $Peak :$' + f' {np.round(p,4)}\n'
        title += r' $Gaussian_{\mu}: $' + f' {np.round(model.mean.value,4)} | '
        title += r' $Gaussian_{\sigma}: $' + f' {np.round(model.stddev.value,4)}'
        

        ax[1].set_title(title, fontsize=15)
        ax[1].set_ylabel('Counts', fontsize=15)

        ax[1].set_xlabel(f'{filt1.upper()}-{filt2.upper()}')

        trgb_cols[i,j,0] = p   # Peak
        trgb_cols[i,j,1] = mean # Bi weighted mean
        trgb_cols[i,j,2] = np.mean(dat) # Mean color
        trgb_cols[i,j,3] = median # Bi weighted median
        
        trgb_cols[i,j,4] = model.mean.value # Gaussian Mean
        trgb_cols[i,j,5] = model.stddev.value
        trgb_cols[i,j,6] = len(dat)
        ax[1].legend(['Data','Gaussian Fit',f'Biweighted Mean of {len(dat)}'])
        #fig.savefig(f"TRGB/reg{k}_color.png")
        #plt.close(fig)
        k+=1
        break
    break

In [ ]:
len(dat)

In [ ]:
x[find_peaks(y,height=12)[0]]

In [ ]:
mean, median, np.median(dat)

In [ ]:
np.save('TRGB/trgb_cols_JK.npy', trgb_cols)

In [ ]:
trgb_cols = np.load('TRGB/trgb_cols_JK.npy')

In [ ]:
plt.figure(figsize=(32,10))
img = plt.imshow(trgb_cols.T[2], cmap='jet_r')
cb = plt.colorbar(img, pad=0.02)
cb.set_label('TRGB Color: Median')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
magss = []
colss = []

filt1 = 'F115Wmag'
filt2 = 'F200Wmag'
filt3 = 'F115Wmag'

for age in [9.6,9.8,10.00001]:
    mags = []
    cols = []
    for met in [0.002,0.003,0.004, 0.006,0.008]:
        df = df_cmd_jwst[ (df_cmd_jwst['logAge']==age) & (df_cmd_jwst['Zini']==met) & (df_cmd_jwst['label']<=3)]

        t = df[df[filt3]==df[filt3].min()]
        mag = t[filt3].values[0]
        col = (t[filt1] - t[filt2]).values[0]
        mags.append(mag)
        cols.append(col)
    magss.append(mags)
    colss.append(cols)

mags = np.array(magss)
cols = np.array(colss)

In [ ]:
fig, ax = plt.subplots()
av_mag = Av_dict[filt3[:5].lower()]*0.19
av_col = (Av_dict[filt1[:5].lower()]-Av_dict[filt2[:5].lower()])*0.19

init = models.Polynomial1D(3)
fit = fitting.LevMarLSQFitter()
x = cols.ravel() + av_col
y = mags.ravel() + av_mag

#model = fit(init,x,y)

ax.plot(cols.T + av_col, (mags.T + av_mag) + 29.729,'-o')


x_new = np.linspace(1.25,1.55)
#ax.plot(x_new, model(x_new),'--k')

#ax.scatter([1.385,1.415,1.455],[23.295]*3,marker='x', color='black')
ax.invert_yaxis()
ax.legend([f'Log(Age) : {i}' for i in [9.6,9.8,10.00001]])
ax.set_xlabel(f'{filt1.upper()}-{filt2.upper()}')
ax.set_ylabel(f'{filt3.upper()}')

In [ ]:
fig, ax = plt.subplots()
"""
0: Peak
1: Bi weighted mean
2: Mean color
3: Bi weighter median
4: Gaussian Mean
"""
ind_ = 3
for i in range(9):  
    if i not in [7,8,9]:
        x = trgb_cols.T[ind_][:,i]
        y = trgb_mags.T[0][:,i]

        ind = np.where((~np.isnan(x) & (x>0.1)))[0]

        x = x[ind]
        y = y[ind]

    ax.scatter(x,y, label='_no_legend_')

ax.plot(cols.T + av_col, (mags.T + av_mag) + 29.719,'-x')

ax.legend([f'Log(Age) : {i}' for i in [ 9.6,9.8,10.00001]])


ax.set_xlabel(f'TRGB ({filt1[:5].upper()} - {filt2[:5].upper()})')
ax.set_ylabel(f'TRGB {filt3[:5].upper()}')

ax.invert_yaxis()

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
ax.tick_params(which='minor', length=4)

Av_y = 26.3
Av_x = 1.45
Av_ = 0.1
AF1_ =  Av_dict[filt1[:5].lower()]*Av_
AF2_ =  Av_dict[filt2[:5].lower()]*Av_
AF3_ =  Av_dict[filt3[:5].lower()]*Av_

dx = AF1_ - AF2_
dy = AF3_

ax.annotate('', xy=(Av_x, Av_y),
             xycoords='data',
             xytext=(Av_x+dx, Av_y+dy),
             textcoords='data',
             arrowprops=dict(arrowstyle= '<|-',
                             color='black',
                             lw=0.5,
                             ls='-')
           )

## **Carbon Stars**

In [ ]:
tab = Table.read('../PHOT_OUTPUT_r40.5/galaxy/f115w_f200w.fits')

In [ ]:
tab['f115w_f200w'] = tab['mag_vega_F115W'] - tab['mag_vega_F200W']

x = np.linspace(1,4)
y = (24-29)*(x-2.1) + 24

init = models.Linear1D()
fit = fitting.LinearLSQFitter()
model_1 = fit(init, x,y)

y = -(24-24.2)*(x-2.5) + 24

model_2 = fit(init, x,y)

ra_cen, dec_cen = 24.1739654, 15.7837025

tab['r'] = angular_separation(tab['ra_1']*u.deg, tab['dec_1']*u.deg, ra_cen*u.deg, dec_cen*u.deg).to(u.arcsec).value

x = tab['ra_1']
y = tab['dec_1']
c = tab['r']

rs = [0, 10]
A  = np.pi*rs[-1]**2

while rs[-1] < 75:
    r = np.sqrt((A+np.pi*rs[-1]**2)/np.pi)
    
    rs.append(r)
    
n_cs = [0]
for i in range(len(rs)-1):
    r_in = rs[i]
    r_out = rs[i+1]
    
    # Annulus
    t = tab[(tab['r']>=r_in) & (tab['r']<r_out)]
    
    # Carbon stars
    t = t[t['mag_vega_F200W']>=model_1(t['f115w_f200w'])]
    t = t[t['mag_vega_F200W']<=model_2(t['f115w_f200w'])]
    
    x = t['ra_1']
    y = t['dec_1']
    
    n_cs.append(len(t))

In [ ]:

fig, ax = plt.subplots()

x = tab['f115w_f200w']
y = tab['mag_vega_F200W']
ax.scatter(x,y, s=0.5)

x = t['f115w_f200w']
y = t['mag_vega_F200W']

ax.scatter(x,y, s=0.5)
x = np.linspace(1,4)
y1 = model_1(x)
y2 = model_2(x)

ax.plot(x,y1,'--r')
ax.plot(x,y2,'--r')

ax.invert_yaxis()
ax.set_xlabel('F115W-F200W')
ax.set_ylabel('F200W')

ax.legend(['Total','Carbon Stars'])

In [ ]:
x = rs
y = n_cs

plt.plot(x,y,'-ok')
plt.xlabel('Radius (arcsecs)')
plt.ylabel('No of Carbon stars')

In [ ]:
t = tab

# Carbon stars
t = t[t['mag_vega_F200W']>=model_1(t['f115w_f200w'])]
t = t[t['mag_vega_F200W']<=model_2(t['f115w_f200w'])]

x = t['ra_1']
y = t['dec_1']

scale = 11417/4486
l = 5
fig, ax = plt.subplots(figsize=(l*scale+3,l))

theta = np.pi/2.75

x,y = x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta)
xmin, xmax = -4.259903639039913, -4.369990102024565
ymin, ymax = 28.524099525386, 28.568378781292367

# Peform the kernel density estimate
xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])

kernel = gaussian_kde(values, bw_method=200/3600)
f = np.reshape(kernel(positions), xx.shape)

f = np.fliplr(f).T

img = ax.contourf(f, cmap='jet', levels=np.linspace(200,f.max(),10),
          extent=[xmax, xmin,
                  ymin, ymax], zorder=200, alpha=0.3)

df_f = df[(df['age']>7) &(df['age']<=9.5)]
x = df_f['RA']
y = df_f['DEC']
c = df_f['age']
scale = 11417/4486
l = 5
theta = np.pi/2.75
ax.scatter(x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta),s=0.2,c=c, cmap='Greys',
                zorder=199)
ax.invert_xaxis()
ax.set_xlim(-4.259903639039913, -4.369990102024565)
ax.set_ylim(28.524099525386, 28.568378781292367)
ax.axis('Off')
cb = plt.colorbar(img, shrink=0.9, pad=-0.03)
cb.set_label('Gaussian Kernel Density')
fig.tight_layout()

#### **Simulated Carbon Star Image**

In [ ]:
tab = Table.read('Carbon_stars.fits')

In [ ]:
df = tab[['ra_2', 'dec_2', 'flux_2','mag_vega_F200W']].to_pandas()

In [ ]:
df = df.rename(columns={'ra_2':'ra',
                'dec_2':'dec',
                'flux_2': 'flux',
                'mag_vega_F200W':'mag'})

In [ ]:
zero_flux = df['flux']/pow(10,-0.4*df['mag'])
zero_flux = np.median(zero_flux)

In [ ]:
2.5*np.log10(zero_flux)

In [ ]:
hdul_n = fits.open(dict_images[det_n][filt_n]['images'][0])
data_n = hdul_n[1].data

pixel_scale = hdul_n[1].header['CDELT1']*3600

wcs_n = WCS(hdul_n[1].header)

In [ ]:
hdul_n[1].header

In [ ]:
wcs_n

In [ ]:
np.arccos(0.422540420042849)*180/np.pi

In [ ]:
psf = fits.open(f'../data/PSF/epsf/F200W/snap_test_psf.fits')[0].data[0,0]
psf = psf.reshape(51,5,51,5).sum(axis=(1,3))
psf /= psf.sum()

hdu = fits.PrimaryHDU(psf)
hdul = fits.HDUList([hdu])
hdul.writeto('psf.fits', overwrite=True)

tel_params ={
            'aperture'       : 650,
            'pixel_scale'    : pixel_scale,
            'psf_file'       : f'psf.fits',
            'response_funcs' :  [ f'{data_path}/INSIST/UV/Coating.dat,5,100',   # 6 mirrors
                                ],
             'coeffs'        : 1/11112.266256569348,
             'theta'         : 65.00492096781794,
             'M_sky' : 27,
            }

In [ ]:
sim = pt.Imager(df, coords=(24.17399193760206,  15.78396038669509),
                tel_params=tel_params, n_x=11604, n_y=4630, exp_time = 2920.4)

In [ ]:
2.5*np.log10(sim.zero_flux), 

In [ ]:
sim.shot_noise = False
sim.sky = False
sim.Bias = False
sim.QN = False
sim.DC = False
sim.PRNU = False
det_params = {'shot_noise' :  'Poisson',
              'qe_response': [],
              'qe_mean'    : 1,
              'G1'         :  1,
              'bias'       : 1,
              'PRNU_frac'  :  0.25/100,
              'DCNU'       :  0.1/100,
              'RN'         :  0.5,
              'T'          :  218,
              'DN'         :  0.01/100
              }

sim(det_params=det_params, photometry=None)

In [ ]:
sim.show_image()

In [ ]:
sim.writeto('../data/pista_carbon_stars_II.fits')

## **Age gradient**

In [ ]:
fs = glob.glob('data/*.csv')
df = pd.read_csv(fs[0])

for f in fs[1:]:
    df_n = pd.read_csv(f)
    df = pd.concat([df, df_n])

In [ ]:
df.keys()

In [ ]:
df_f = df[(df['age']>5.9) &(df['age']<=7)]
x = df_f['RA']
y = df_f['DEC']
c = df_f['age']
scale = 11417/4486
l = 5
fig, ax = plt.subplots(figsize=(l*scale+3,l))

theta = np.pi/2.75
img = ax.scatter(x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta),s=5,c=c, cmap='jet')
#ax.set_xlabel('RA')
#ax.set_ylabel('DEC')
ax.invert_xaxis()
ax.set_xlim(-4.259903639039913, -4.369990102024565)
ax.set_ylim(28.524099525386, 28.568378781292367)
ax.axis('Off')
cb = plt.colorbar(img, shrink=0.9, pad=-0.03)
cb.set_label('log(Age)')
fig.tight_layout()

In [ ]:
df_f = df[(df['age']>7) &(df['age']<=8)]
x = df_f['RA']
y = df_f['DEC']
c = df_f['age']
scale = 11417/4486
l = 5
fig, ax = plt.subplots(figsize=(l*scale+3,l))

theta = np.pi/2.75
img = ax.scatter(x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta),s=0.2,c=c, cmap='jet')
#ax.set_xlabel('RA')
#ax.set_ylabel('DEC')
ax.invert_xaxis()
ax.set_xlim(-4.259903639039913, -4.369990102024565)
ax.set_ylim(28.524099525386, 28.568378781292367)
ax.axis('Off')
cb = plt.colorbar(img, shrink=0.9, pad=-0.03)
cb.set_label('log(Age)')
fig.tight_layout()

In [ ]:
df_f = df[(df['age']>8) &(df['age']<=9.6)]
x = df_f['RA']
y = df_f['DEC']
c = df_f['age']
scale = 11417/4486
l = 5
fig, ax = plt.subplots(figsize=(l*scale+3,l))

theta = np.pi/2.75
img = ax.scatter(x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta),s=0.2,c=c, cmap='jet')
#ax.set_xlabel('RA')
#ax.set_ylabel('DEC')
ax.set_xlim(-4.259903639039913, -4.369990102024565)
ax.set_ylim(28.524099525386, 28.568378781292367)
ax.invert_xaxis()
ax.axis('Off')
cb = plt.colorbar(img, shrink=0.9, pad=-0.03)
cb.set_label('log(Age)')
fig.tight_layout()

In [ ]:
df_f = df[(df['age']>9.6) &(df['age']<=10)]
x = df_f['RA']
y = df_f['DEC']
c = df_f['age']
scale = 11417/4486
l = 5
fig, ax = plt.subplots(figsize=(l*scale+3,l))

theta = np.pi/2.75
img = ax.scatter(x*np.cos(theta) - y*np.sin(theta),x*np.sin(theta) + y*np.cos(theta),s=0.2,c=c, cmap='jet')
#ax.set_xlabel('RA')
#ax.set_ylabel('DEC')
ax.invert_xaxis()
ax.axis('Off')
cb = plt.colorbar(img, shrink=0.9, pad=-0.03)
cb.set_label('log(Age)')
fig.tight_layout()

## **HST**

### **PARSEC Isochrones**

#### **ACS/WFC**

In [ ]:
with open("../data/isochrones_master/cmd_hst_acs.dat") as f:
    dat = f.readlines()

data = []

for i,d in enumerate(dat[15:-1]):
    if 'Zini' not in d:
        data.append([float(i) for i in d.split()])
        
df_cmd_acs = pd.DataFrame(data,columns=dat[13][2:].split())[:-1]
#df_cmd_acs = df_cmd_acs[df_cmd_acs['Zini']==0.02]
df_cmd_acs.to_csv("../data/isochrones_master/cmd_hst_acs.csv", index=None)

#df_cmd_acs = df_cmd_acs[(df_cmd_acs['label']>-1) & (df_cmd_acs['label']<7)]

In [ ]:
df_cmd_acs= pd.read_csv("../data/isochrones_master/cmd_hst_acs.csv")

In [ ]:
ages = np.unique(df_cmd_acs['logAge'])
mets = np.unique(df_cmd_acs['Zini'])
mag_err_lim = 0.2

In [ ]:
mets

In [ ]:
if not os.path.exists('../data/isochrones/HST_ACS'):
    os.mkdir('../data/isochrones/HST_ACS/')
else:
    os.system('rm ../data/isochrones/HST_ACS/*')
    
for i, age in enumerate(ages):
    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)
    for met in [0.02, 0.008]:
        
        temp = df_cmd_acs[df_cmd_acs['logAge']==age]
        temp = temp[temp['Zini']==met]
        temp = temp[['label', 'Mini', 'F435Wmag', 'F555Wmag', 'F814Wmag', 'Zini', 'logAge']]
        temp =  temp[temp['Mini']>=2]
        header = list(temp.keys())
        header[0] = '#' + header[0]

        temp.to_csv(f'../data/isochrones/HST_ACS/{i}_PARSEC1.1_Z{met}_logAGE{age}Myr_HST_BVI.isoc',sep=' ',
                   index=None, header = header)

#### **WFC3/UVIS**

In [ ]:
with open("../data/isochrones_master/cmd_hst_uvis.dat") as f:
    dat = f.readlines()

data = []

for i,d in enumerate(dat[13:-1]):
    if 'Zini' not in d:
        data.append([float(i) for i in d.split()])
        
df_cmd_uvis = pd.DataFrame(data,columns=dat[13][2:].split())[:-1]
df_cmd_uvis.to_csv("../data/isochrones_master/cmd_hst_uvis.csv")
df_cmd_uvis = df_cmd_uvis[df_cmd_uvis['MH']==0.51508]
#df_cmd_uvis = df_cmd_uvis[(df_cmd_uvis['label']>-1) & (df_cmd_uvis['label']<7)]

In [ ]:
df_cmd_uvis = pd.read_csv("../data/isochrones_master/cmd_hst_uvis.csv")

In [ ]:
ages = np.round(np.arange(6.0,8.1,0.1),1)

mag_err_lim = 0.2

In [ ]:
if not os.path.exists('../data/isochrones/HST_UVIS'):
    os.mkdir('../data/isochrones/HST_UVIS/')
else:
    os.system('rm ../data/isochrones/HST_UVIS/*')
    
for i, age in enumerate(ages):
    temp = df_cmd_uvis[df_cmd_uvis['logAge']==age]
    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)
    
    temp = temp[['label', 'Mini', 'F275Wmag', 'F336Wmag', 'Zini', 'logAge']]
    temp =  temp[temp['Mini']>=2]
    header = list(temp.keys())
    header[0] = '#' + header[0]
        
    temp.to_csv(f'../data/isochrones/HST_UVIS/{i}_PARSEC1.1_Z0.015_logAGE{age}Myr_HST_UVIS.isoc',sep=' ',
               index=None, header = header)

### **CMD**

In [ ]:
ages_ = np.round(np.log10([ 7e6, 1e7, 2e7, 3e7, 5e7,1e8]),1)

In [ ]:
age_lin = []
for i in ages_:
    if i > 6  and i <9:
        i-=6
        age_lin.append(f'{np.ceil(10**i)} Myr')
    elif i >= 9:
        i-=9
        age_lin.append(f'{np.ceil(10**i)} Gyr')
        

In [ ]:
r_in = 0
r_out = 500

fig, ax, tab = gen_CMD('f435w','f555w', name = 'f435w_f555w',cmd=df_cmd_acs, out_dir='photometry/ngc628',
                 Av_=0.5, xlims=[-2,3], ylims=[20,28], r_in=r_in,r_out=r_out, ref_xpos = -1.5,
                label = 9, met=0.02, regions=['filament'], ages=ages_, dismod=29.75,
                  Av_x = 2, Av_y=27)

In [ ]:
r_in = 0
r_out = 500

fig, ax, tab = gen_CMD('f555w','f814w', name = 'f555w_f814w',cmd=df_cmd_acs, out_dir='photometry/ngc628',
                 Av_=0.5, xlims=[-2,6], ylims=[20,28], r_in=r_in,r_out=r_out, ref_xpos = -1.5,
                label = 3, met=0.02, regions=['bubble'], ages=[9.0,9.4,9.8,10,10.1], dismod=29.75, gen_kde=True,
                  Av_x = 2, Av_y=27)

In [ ]:
r_in = 0
r_out = 12

fig, ax = plt.subplots(1,2, sharey=False,figsize=(24,10))

filt1 = 'f435w'

filt2 = 'f814w'

filt3 = 'f814w'

fig,ax1, tab = gen_CMD(filt1, filt2,filt3, name = 'ngc628/f435w_f814w',
                       cmd=None, Av_=1, Av_x=4, Av_y=26, xlims=[-1,6], ylims=[19,28],
                     out_dir='photometry',gen_kde=True,gen_contours=False, label=10, met=0.02,
                    dismod=29.8, ages=ages_, sqr_field=False, r_in=r_in, r_out=r_out, regions=['bubble'],
                    fig=fig,ax=ax[0],ref_xpos=-0.5)

fig,ax2, tab = gen_CMD(filt1, filt2,filt3, name = 'ngc628/f435w_f814w',
                       cmd=None, r=None, Av_=1, Av_x=4, Av_y=26, xlims=[-1,6], ylims=[19,28],
                 out_dir='photometry',gen_kde=False, label=10, met=0.02,dismod=29.95, ages=ages_,
                 r_in=r_in, r_out=r_out, fig=fig,ax=ax[1],ref_xpos=-0.5, alpha=0.5, s=0.5,
                regions=['bubble'])


In [ ]:
r_in = 0
r_out = 500

fig,ax1, tab = gen_CMD('f435w', 'f814w', 'f814w',name = 'ngc628/f435w_f814w',
                       cmd=df_cmd_acs, r=None, Av_=1, Av_x=1.5, Av_y=27, xlims=[-0.5,6], ylims=[17,30],
                 out_dir='photometry',gen_kde=True, label_min=0, label_max=10, met=None,dismod=30.04, ages=[8],
                 r_in=r_in, r_out=r_out,regions=['galaxy'])
#ax1.set_title('')
#ax1.get_legend().remove() 

In [ ]:
x = tab['mag_vega_F814W']
y = tab['mag_err_2']

fig, ax = plt.subplots()

ax.scatter(x,y,s=0.05, color='black')

ax.set_xlabel('F814W (vega)')
ax.set_ylabel('mag_err')

ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=7,direction="in", bottom=True, top=True,left=True, right=True)
ax.tick_params(which='minor', length=4)

In [ ]:
x = tab['ra_1']
y = tab['dec_1']

plt.scatter(x,y, s=0.1)

In [ ]:
r_in = 0
r_out = 10

fig, ax, tab = gen_CMD('f435w','f814w', name = 'f435w_f814w',cmd=df_cmd_acs, out_dir='photometry/ngc628',
                 Av_=0.5, xlims=[-2,6], ylims=[18,28], r_in=r_in,r_out=r_out, ref_xpos = -1.5,
                label = 3, met=0.02, regions=['bubble'], ages=ages_, dismod=29.8,
                  Av_x = 2, Av_y=27)

In [ ]:
ages_ = np.round(np.log10([4e6, 7e6, 1e7, 2e7, 3e7, 5e7]),1)

In [ ]:
ages_

In [ ]:
age_lin = []
for i in ages_:
    if i > 6  and i <9:
        i-=6
        age_lin.append(f'{np.ceil(10**i)} Myr')
    elif i >= 9:
        i-=9
        age_lin.append(f'{np.ceil(10**i)} Gyr')
        

In [ ]:
fig, ax = gen_CMD('f275w','f336w', cmd=df_cmd_uvis, out_dir='PHOT_OUTPUT_r25',
                 Av_=0.5, xlims=[-3,5], ylims=[18,28], r=10/0.04, met=0.008,label=9, ref_xpos=-2,
                 x_cen=631/2, y_cen=631/2, regions=['bubble','bkg3'], Av_x=4, Av_y=24,
                 flag=111)

### **Extinction Vector**

In [ ]:
df_cmd_hst['Zini']

## **JWST-HST**

In [ ]:
df_cmd_hst_jwst = pd.read_csv('../data/isochrones_master/cmd_jwst_hst.csv')

In [ ]:
ages = ages = np.round(np.arange(6.0,9.1,0.1),1)

In [ ]:
if not os.path.exists('../data/isochrones/HST_JWST'):
    os.mkdir('../data/isochrones/HST_JWST/')
else:
    os.system('rm ../data/isochrones/HST_JWST/*')
    
for i, age in enumerate(ages):
    temp = df_cmd_hst_jwst[df_cmd_hst_jwst['logAge_1']==age]
    if len(str(i+1))==1:
        i = '0' + str(i+1)
    else:
        i =  str(i+1)
    
    temp = temp[['label_1', 'Mass_1', 'F435Wmag', 'F555Wmag', 'F814Wmag', 'F115Wmag', 'F150Wmag', 'F200Wmag', 'Zini_1', 'logAge_1']]
    header = list(temp.keys())
    header[0] = '#' + header[0]
        
    temp.to_csv(f'../data/isochrones/HST_JWST/{i}_PARSEC1.1_Z0.02_logAGE{age}Myr_JWST_JHK.isoc',sep=' ',
               index=None, header = header)


In [ ]:
r_in = 0
r_out = 10

fig,ax = gen_CMD('f200w', 'f555w', name='f115w_f150w_f200w_f435w_f555w_f814w',
                 cmd=df_cmd_hst_jwst, r=None, ra_col='ra_1_1', dec_col='dec_1_1',
                 Av_=3, Av_x=2.5, Av_y=26, xlims=[-7,5], ylims=[18,30],
                 out_dir='PHOT_OUTPUT_r25',gen_kde=False, label=9, met=0.02,
                 r_in=r_in, r_out=r_out,
                regions=['bubble'])

## **SED**

In [ ]:
df_sed_bub = Table.read("../data/f275_f336w_f435w_f336w_f115w_f150w_f200w_bub.fits")
df_sed_bkg = Table.read("../data/f275_f336w_f435w_f336w_f115w_f150w_f200w_bkg.fits")

In [ ]:
len(df_sed_bub), len(df_sed_bkg)

In [ ]:
filts = [i.upper() for i in list(Av_dict.keys())]
filts_wav = [2709.7, 3354.5, 4322.0, 5368.0, 8146.9, 11490, 14970, 19680]
Avs = np.array(list(Av_dict.values()))

In [ ]:
m = 21
dm = 0.5

df_filt1 = df_sed_bub[(df_sed_bub['mag_vega_F115W']>= m) 
                      & (df_sed_bub['mag_vega_F115W']<= m + dm)]

df_filt2 = df_sed_bkg[(df_sed_bkg['mag_vega_F115W']>= m) 
                      & (df_sed_bkg['mag_vega_F115W']<= m + dm)]

fig, ax = plt.subplots(figsize=(15,8))

for i in range(len(df_filt1)):

    y = df_filt1[i:i+1]['mag_vega_F275W', 'mag_vega_F336W', 'mag_vega_F435W', 'mag_vega_F555W', 'mag_vega_F814W',
                'mag_vega_F115W', 'mag_vega_F150W', 'mag_vega_F200W'].to_pandas().values[0]

    ax.plot(filts_wav,y,'-ob')
 

for i in range(len(df_filt2)):

    y = df_filt2[i:i+1]['mag_vega_F275W', 'mag_vega_F336W', 'mag_vega_F435W', 'mag_vega_F555W', 'mag_vega_F814W',
                'mag_vega_F115W', 'mag_vega_F150W', 'mag_vega_F200W'].to_pandas().values[0]

    #ax.plot(filts_wav,y,'-or')
    
ax.xaxis.set_major_locator(AutoLocator())
ax.xaxis.set_minor_locator(AutoMinorLocator())

ax.yaxis.set_major_locator(AutoLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', length=10,direction="in", bottom=True, top=True,left=True, right=True)
ax.tick_params(which='minor', length=4);
ax.set_xticks(filts_wav, labels=filts, rotation=45, fontsize = 15)
ax.invert_yaxis();

## **CMD gradient**

In [ ]:
rs = [0,5/0.031]

area = np.pi*(rs[-1]**2 - rs[-2]**2)

while 1:
    r = np.sqrt(area/np.pi + rs[-1]**2)
    if r < 797/2:
        rs.append(r)
    else:
        break
    
rs = np.array(rs)

In [ ]:
rs

In [ ]:
rs[1:]**2 - rs[:-1]**2

In [ ]:
x_cen = 797/2
y_cen = 797/2

In [ ]:
plt.figure(figsize=(7, 10))

ax1 = plt.subplot(1, 1, 1,projection=cutout_n.wcs)

ax1.set_title(filt_n, fontdict=font2)
norm = simple_norm(cutout_n.data, 'sqrt',percent=99.)

img = ax1.imshow(cutout_n.data, norm = norm, cmap='gray')

levels = [13.2,15]

smooth_data = gaussian_filter(cutout_m.data, 2)
cmap = ListedColormap(['yellow','cyan'])
ax1.contour(smooth_data, levels=levels, cmap=cmap, transform=ax1.get_transform(cutout_m.wcs))
#ax1.grid(color='red')


for r in rs:
    circle = Circle((x_cen, y_cen), r, fill=False,  color='black')
    ax1.add_patch(circle)
 
circle = Circle((x_cen, y_cen), 200, fill=False,  color='red')
ax1.add_patch(circle)

circle = Circle((x_cen, y_cen), 282.843, fill=False,  color='red')
ax1.add_patch(circle)

circle = Circle((x_cen, y_cen), rs.max(), fill=False,  color='red')
ax1.add_patch(circle)
    
cb = plt.colorbar(img, orientation='horizontal', anchor = (0.5, 1.1))
cb.set_label(f"{imh_m['BUNIT']}")

In [ ]:
mag_err_lim = 0.1

In [ ]:
age_lin = []
for i in ages_:
    if i > 6  and i <9:
        i-=6
        age_lin.append(f'{np.round(10**i,2)} Myr')
    elif i >= 9:
        i-=9
        age_lin.append(f'{np.round(10**i,2)} Gyr')   

In [ ]:
for i in range(len(rs)-1):
    fig,ax = gen_CMD('f115w', 'f150w', r_in=rs[i],r_out=rs[i+1],
                     cmd=None, flag=111, gen_kde=False, add_ref=True,
                     x_cen=797/2, y_cen=797/2, out_dir='PHOT_OUTPUT_r25')
    fig.suptitle(f"R = {np.round(rs[i],1)}-{np.round(rs[i+1],1)}", fontsize=20)
    plt.show()

## **Simulate Stars using PISTA**

In [ ]:
region, filt_n

In [ ]:
filt_n = 'F435W'

In [ ]:
if not os.path.exists(f"../PHOT_OUTPUT_m50/{region}/{filt_n}"):
    os.mkdir(f"../PHOT_OUTPUT_m50/{region}/{filt_n}")

In [ ]:
hdu = fits.open(f"../PHOT_OUTPUT_AUTO/{region}/{filt_n}/data_conv.fits")[1]

In [ ]:
data_source = hdu.data
pixel_scale = hdu.header['CDELT1']*3600

In [ ]:
norm = simple_norm(data_source, 'sqrt', percent=99.)
plt.imshow(data_source, cmap='jet', norm=norm)

In [ ]:
phot_table = Table.read(f"../PHOT_OUTPUT_AUTO/{region}/{filt_n}/{filt_n}_photometry_filt.fits")

In [ ]:
psf = np.median(fits.open(f'../data/PSF/epsf/{filt_n}/snap_test_psf.fits')[0].data, axis=(1,0))
psf = psf.reshape(51,5,51,5).sum(axis=(1,3))
#psf = np.where(psf>=0,psf,0)
#psf -= psf.min()
psf /= psf.sum()

In [ ]:
hdu = fits.PrimaryHDU(psf)
hdul = fits.HDUList([hdu])
hdul.writeto('psf.fits', overwrite=True)

In [ ]:
data = psf
norm = simple_norm(data, 'log', percent=99.99, log_a=100)
img = plt.imshow(data, cmap='jet', norm=norm)
plt.colorbar(img)

In [ ]:
tel_params ={
            'aperture'       : 650,
            'pixel_scale'    : pixel_scale,
            'psf_file'       : f'psf.fits',
            'response_funcs' :  [ f'{data_path}/INSIST/UV/Coating.dat,5,100',   # 6 mirrors
                                ],
             'coeffs'        : 0.5 ,
             'theta'         : 0
            }

In [ ]:
df = phot_table[ (phot_table['SNR']<20) ][['ra', 'dec',f'mag_vega_{filt_n}', 'flux','flag_phot']].to_pandas()

In [ ]:
df = df[df['flag_phot']==1]

In [ ]:
zero_flux = (df['flux']/10**(-0.4*df[f'mag_vega_{filt_n}'])).mean()

In [ ]:
zp = 2.5*np.log10(zero_flux)

In [ ]:
zero_flux = 10**(0.4*(zp+0.5))

In [ ]:
df = df.rename(columns = {f'mag_vega_{filt_n}': 'mag'})

In [ ]:
sim = pt.Imager(df=df, tel_params=tel_params, exp_time=2000,
               n_x=data_source.shape[0], n_y=data_source.shape[1])

In [ ]:
det_params = {'shot_noise' :  'Poisson',
              'qe_response': [],
              'qe_mean'    : 1,
              'G1'         :  1,
              'bias'       : 10,
              'PRNU_frac'  :  0.25/100,
              'DCNU'       :  0.1/100,
              'RN'         :  3,
              'T'          :  218,
              'DN'         :  0.01/100
              }

In [ ]:
(sim.n_x_sim - sim.n_x)/2, sim.n_x_sim - (sim.n_x_sim - sim.n_x)/2

In [ ]:
sim.shot_noise=False

In [ ]:
sim(det_params=det_params)

In [ ]:
sim.digital.min()

In [ ]:
sim.show_image()

In [ ]:
x = np.random.uniform(100.0,sim.n_x_sim -100,3600)
y = np.random.uniform(100.0,sim.n_y_sim -100,3600)

mag = np.array(list(np.linspace(20,32,60))*60)
df_add  = pd.DataFrame(zip(x,y,mag), columns = ['x','y','mag'])

In [ ]:
sim.shot_noise = False

In [ ]:
out_img = sim.add_stars(data_source, zero_flux, df_add)

In [ ]:
norm = simple_norm(out_img, 'sqrt', percent=99.)
plt.imshow(out_img, cmap='jet', norm=norm)

In [ ]:
hdu = fits.open(f"../PHOT_OUTPUT_EPSF/{region}/{filt_n}/data_conv.fits")

In [ ]:
hdu[1].data = out_img
wcs = WCS(hdu[1].header)

In [ ]:
coords = np.array(wcs.array_index_to_world_values(y-49,x-49))

In [ ]:
df_add['x'] = x - 49
df_add['y'] = y - 49

In [ ]:
df_add['ra'] = coords[0,:]
df_add['dec'] = coords[1,:]

In [ ]:
df_add.to_csv(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/add_stars.csv", index=None)

In [ ]:
hdu.writeto(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/data_conv.fits", 
            overwrite=True)

## **Fitting Pritchet Function**

In [ ]:
@models.custom_model
def pritchet(m,alpha=0.5,m_50=30):
    return 0.5*(1 - alpha*(m - m_50)/np.sqrt(1 + alpha**2*(m-m_50)**2))

In [ ]:
df_match = pd.read_csv(f"../PHOT_OUTPUT_m50/{region}/{filt_n}/matched.csv")

In [ ]:
x = df_match['mag']
y = df_match[f'mag_vega_{filt_n}'] 
yerr = df_match['mag_err']

plt.figure(figsize=(5,5))
plt.errorbar(x, y, yerr, fmt='o', color = 'black', markersize=1, capsize=2)
plt.xlabel('Mag in')
plt.ylabel('Mag out')
plt.plot([25,30],[25,30],'--r')

In [ ]:
df_match = df_match[df_match['flag_phot']==1]
df_match = df_match[abs(df_match['mag'] - df_match[f'mag_vega_{filt_n}'])<2]

In [ ]:
x = df_match['mag']
y = df_match[f'mag_vega_{filt_n}']
yerr = df_match['mag_err']

plt.figure(figsize=(5,5))
plt.errorbar(x, y, yerr, fmt='o', color = 'black', markersize=1, capsize=2)
plt.plot([25,30],[25,30],'--r')

In [ ]:
x = np.array(df_match.groupby('mag').count().index)
y = np.array(df_match.groupby('mag').count()['x_1']/60)

y = y[np.where((x>22)&(x<30))[0]]
x = x[np.where((x>22)&(x<30))[0]]
init = pritchet()
fit = fitting.LevMarLSQFitter()
offset = y.max()
model = fit(init, x, y/offset)

fig, ax = plt.subplots(figsize=(9,7))
ax.scatter(x,y)

ax.plot(x, model(x)*offset, '--r')
ax.set_xlabel('mags')
ax.set_ylabel(r'$N_{out}/N_{in}$')
ax.set_title(f"{region} | {filt_n} | "+ r"$\alpha =$" + f" {np.round(model.alpha.value,2)}" + r" | $m_{50}=$" + f"{np.round(model.m_50.value,2)}")

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(which='both', width=2,direction="in", top = True,right = True,
               bottom = True, left = True)
ax.tick_params(which='major', length=7,direction="in")
ax.tick_params(which='minor', length=4, color='black',direction="in")
ax.set_ylim(0,1)

bkg2, F115W 25.59

In [ ]:
float(np.array(1))

In [ ]:
y